In [137]:
# !pip install tensorflow networkx pysmiles deepchem
from pysmiles import read_smiles
from networkx.algorithms.approximation import treewidth_min_degree
from networkx.algorithms.approximation import treewidth_min_fill_in
from networkx.algorithms import junction_tree
import networkx as nx
import itertools
import math
import copy
import time
import functools
import re
import logging
logging.getLogger('pysmiles').setLevel(logging.CRITICAL)  # Anything higher than warning
# !pip install matplotlib
# import matplotlib

In [66]:
pcm = read_smiles("CC(=O)Nc1ccc(O)cc1")

In [67]:
def remap_graph(graph: nx.Graph) -> nx.Graph:
  """Takes an input graph and remaps it such that the old key/value pair of a node turns into (integer, old key)
  """
  
  #remap nodes
  node_mapping = {}
  iter = 0
  for node in graph.nodes:
    # print(f"In remap, setting {node} equal to {iter}")
    node_mapping[node] = iter
    iter += 1

  #remap edges
  new_edges = []
  for pair in graph.edges:
    new_edges.append((node_mapping[pair[0]],node_mapping[pair[1]]))
  output_graph = nx.Graph()
  for node in node_mapping:
    # print(node,node_mapping[node])
    output_graph.add_node(node_mapping[node],values=list(node))
  output_graph.add_edges_from(new_edges)
  return output_graph

In [68]:

def add_leaves(decomposition):
  """ Adds nice tree decomposition-defined "leaves" to input graph (which is assumed to be a tree decomposition here)

  The definition of a leaf in a nice tree decomposition is that it contains only
  one element as its value. 
  This function finds all terminal vertices in the graph and counts the number
  of values they each contain. If any terminal vertex contains more than 1 value,
  a leaf is created by appending a node to be the new terminal vertex and setting
  the value of this node equal to one value of the previously terminal vertex.
  Returns a new graph with leaves added as well as a new root which is the first leaf added 
  E.g.:   (1,2,3) - (2,3)

  Would construct leaves on both sides as:

  (1,) - (1,2,3) - (2,3) - (2,)
  """

  output = decomposition.copy()
  temporary_root = None
  values = nx.get_node_attributes(decomposition, "values")
  current_id = max(output.nodes)
  for node, degree in list(output.degree()):
    # print(node, degree)
    # TODO: Just delete this? What node will ever have degree == 0??
    if degree == 0 and len(values[node]) > 1:
      current_id += 1
      output.add_node(current_id,values=[])
      output.add_edge(node,current_id)
      current_id += 1
      output.add_node(current_id,values=[])
      output.add_edge(node,current_id)
      if temporary_root == None:
        temporary_root = current_id
      

    elif degree == 1 and len(values[node]) > 1:
      current_id += 1
      output.add_node(current_id,values=[])
      output.add_edge(node,current_id)
      if temporary_root == None:
        # We s
        temporary_root = current_id
  return output, temporary_root

In [69]:
# Constructs junction tree (decomposition), "remaps" the decomposition values
# and adds leaves
def construct_junction_and_add_leaves(graph):
  remapped_and_leaved_graph, root = add_leaves(remap_graph(junction_tree(graph)))
  return remapped_and_leaved_graph, root

In [70]:
def handle_promiscuous_node2(decomposition, promiscuous_node, parent):
  
  new_bag = decomposition.nodes.data("values")[promiscuous_node]
  new_node_id = max(decomposition.nodes)+1
  # print("adding new bag: ", new_bag)
  decomposition.add_node(new_node_id,values=new_bag)
  # print("removing parent: ",parent)
  # print(list(decomposition.neighbors(promiscuous_node)))
  neighbors = list(decomposition.neighbors(promiscuous_node))
  # print("children to update: ",children_to_update)
  if parent != None:
    neighbors.remove(parent)
  children_to_update = neighbors[1::]
  decomposition.add_edge(promiscuous_node,new_node_id)
  for child in children_to_update:
    decomposition.remove_edge(promiscuous_node,child)
    decomposition.add_edge(new_node_id,child)


def handle_promiscuous_nodes2(decomposition,root,parent):
  # print("current root: ",root)
  # print("children: ",children)
  children = list(decomposition.neighbors(root))
  # print("in handle prom. nodes. children: ",children)
  if parent != None:
    children.remove(parent)
  if len(children) > 2:
    handle_promiscuous_node2(decomposition,root,None)
    handle_promiscuous_nodes2(decomposition,root,None)
  else:
    for child in children:
      if len(list(decomposition.neighbors(child))) > 3:
        handle_promiscuous_node2(decomposition,child,root)
      handle_promiscuous_nodes2(decomposition,child,root)


In [71]:
def insert_bridging_node(graph, root, child, new_node_id, new_node_values:list):
  assert(type(new_node_values) == list)
  graph.add_node(new_node_id,values=new_node_values)
  graph.add_edge(root,new_node_id)
  graph.add_edge(new_node_id,child)
  graph.remove_edge(root,child)

In [72]:
def handle_join_node(decomposition,root,children):
  """This function handles the construction of a single join node and its two children.
  The join node is given in the `root` variable and the two children are given in the
  `children` variable.
  The join operation is performed by taking the union of the values of all three
  nodes: the parent and its two children, and then setting the values of all three
  nodes equal to this unioned bag.
  """
  # print("Handling join node: ", root)
  assert(len(children) == 2)
  assert(type(children) == list)
  values = nx.get_node_attributes(decomposition, "values")
  root_values = set(values[root])
  child0_values = set(values[children[0]])
  child1_values = set(values[children[1]])
  if root_values == child0_values == child1_values:
    # print(f"Join node {root} already fulfills join identity")
    return children[0], children[1]
  unioned_bag = set(values[root]) | set(values[children[0]]) | set(values[children[1]])
  new_node_id_1 = max(decomposition.nodes)+1
  new_node_id_2 = new_node_id_1+1
  insert_bridging_node(decomposition, root, children[0], new_node_id_1,list(unioned_bag))
  insert_bridging_node(decomposition, root, children[1], new_node_id_2,list(unioned_bag))
  # print("unioned bag:")
  # print(unioned_bag)
  decomposition.nodes[root]["values"] = list(unioned_bag)
  return new_node_id_1, new_node_id_2
  # decomposition.nodes[children[0]]["values"] = tuple(unioned_bag)
  # decomposition.nodes[children[1]]["values"] = tuple(unioned_bag)

In [73]:
def handle_join_nodes(decomposition,root,parent):
  children = list(decomposition.neighbors(root))
  # print(children)
  if parent != None:
    children.remove(parent)
  if len(children) == 2:
    new_child_1, new_child_2 = handle_join_node(decomposition,root,children)
    handle_join_nodes(decomposition,new_child_1,root)
    handle_join_nodes(decomposition,new_child_2,root)

  else:
    for child in children:
      handle_join_nodes(decomposition,child,root)

In [74]:

# def fix_useless_introducers(decomposition,root):
#     root_child = list(decomposition.neighbors(root))
#     root_child = root_child[0]
#     root_grandchild = list(decomposition.neighbors(root_child))
#     root_grandchild.remove(root)
#     root_grandchild = root_grandchild[0]
#     def inner_recurse(decomposition,current,parent,grandparent):
#         print(f"Recursing on: c, p, gp {current, parent, grandparent}, decomp nodes: {decomposition.nodes}")
#         children = list(decomposition.neighbors(current))
#         if parent == None:
#             inner_recurse(decomposition,children[0],current,grandparent)
#         elif grandparent == None:
#             children.remove(parent)
#             inner_recurse(decomposition,children[0],current,parent)
#         try:
#             children.remove(parent)
#         except:
#             pass

#         if len(children) > 0:
#             current_bag = decomposition.nodes[current]["values"]
#             parent_bag = decomposition.nodes[parent]["values"]
#             grandparent_bag = decomposition.nodes[grandparent]["values"]
#             if len(parent_bag) < len(current_bag) and set(current_bag) == set(grandparent_bag):
#                 decomposition.remove_node(parent)
#                 decomposition.remove_node(current)
#                 print(f"Removed nodes: {current, parent}")
#                 decomposition.add_edge(children[0],grandparent)
#                 inner_recurse(decomposition,children[0],current,None)
#             else:
#                 inner_recurse(decomposition,children[0],current,parent)

#     inner_recurse(decomposition,root_grandchild,root_child,root)


def fix_useless_introducers(decomposition,current,parent,grandparent):
    children = list(decomposition.neighbors(current))
    print(f"Recursing on: c, p, gp {current, parent, grandparent}, decomp nodes: {decomposition.nodes}, children: {children}")
    # if len(children) > 1:
    if parent == None:
        fix_useless_introducers(decomposition,children[0],current,grandparent)
    elif grandparent == None and len(children) >1:
        children.remove(parent)
        fix_useless_introducers(decomposition,children[0],current,parent)
    else:
        try:
            children.remove(parent)
        except:
            pass
        # print(f"Children: {children}")
        if len(children) > 0:
            current_bag = decomposition.nodes[current]["values"]
            parent_bag = decomposition.nodes[parent]["values"]
            grandparent_bag = decomposition.nodes[grandparent]["values"]
            if len(parent_bag) < len(current_bag) and set(current_bag) == set(grandparent_bag):
                decomposition.remove_node(parent)
                decomposition.remove_node(current)
                print(f"Removed nodes: {current, parent}")
                decomposition.add_edge(children[0],grandparent)
                fix_useless_introducers(decomposition,children[0],grandparent,None)
            else:
                fix_useless_introducers(decomposition,children[0],current,parent)

In [142]:
def handle_amnesiac_node(graph, root, child, root_values_set, child_values_set):
  """ This function handles an "amnesiac node": |B_parent| < |B_child|-1

      I.e. where the parent contains 2 or fewer elements than the child does. This
      indicates that the parent has "forgotten too much" in one step (is amnesiac)
  """

  assert type(root_values_set) == set
  assert type(child_values_set) == set

  # Find the "forgotten values", i.e. those values which are present in the bag
  # of the child but not in the bag of the parent 
  forgotten_values = list(child_values_set-root_values_set)
  assert len(forgotten_values) > 1, f"We expect 2 or more elements forgotten, but only got these: {forgotten_values}"
  # print(f"Forget case. Root elements: {root_values_set}, child elements: {child_values_set}")

  # Constructing a new, bridging node.
  # I will arbitrarily pick the first element of the forgotten nodes
  element_to_forget = forgotten_values[0]

  # The bag of the new, bridging node is constructed. This is simply done by appending
  # the arbitrarily chosen forgotten element to the bag of the parent:
  new_node_bag = root_values_set.copy()
  new_node_bag.add(element_to_forget)

  # id of the new node is just obtained by incrementing the current max in the graph by 1
  new_node_id = max(graph.nodes)+1
  # print("adding new forget node: ", new_node_id)
  # print("With the bag: ", new_node_bag)

  # removing the old edge between root and child
  # inserting the new node between the root and child
  insert_bridging_node(graph,root,child,new_node_id,list(new_node_bag))
  return new_node_id



def handle_eager_introducer(graph, root, child, root_values_set, child_values_set):
  """This function handles an eager introducer node: |B_parent| > |B_child|+1

      I.e. where the bag of the parent contains 2 or more elements more than the 
      bag of the child
  """
  
  assert type(root_values_set) == set
  assert type(child_values_set) == set

  # Find the surplus of introduced nodes (i.e. those nodes in the parent bag
  # but not in the child bag)
  introduced_surplus = list(root_values_set-child_values_set)
  element_to_introduce = introduced_surplus[0]

  # The bag of the new bridging node is constructed by removing one element
  # from the bag of the parent node
  new_node_bag = root_values_set.copy()
  new_node_bag.remove(element_to_introduce)
  new_node_id = max(graph.nodes)+1
  # print(f"Calling insert bridgin node with root: {root} and child: {child} with graph edges: {graph.edges}")
  insert_bridging_node(graph, root, child, new_node_id, list(new_node_bag))
  return new_node_id




# def handle_identical(graph,root,child,root_values,child_values):
#   """ This function handles the case where the bags of the parent and child are identical:
#       B_parent = B_child

#       This is handled by creating a bridging node which contains the bag of the parent
#       with one element removed 
#   """
  
#   assert type(root_values) == set
#   assert type(child_values) == set
#   element_to_remove = tuple(root_values)[0]
#   new_node_bag = root_values.copy()
#   new_node_bag.remove(element_to_remove)
#   new_node_id = max(graph.nodes)+1
#   insert_bridging_node(graph,root,child,new_node_id,list(new_node_bag))
#   return new_node_id
    

In [270]:
def correct_unnice_node(graph, current, child, current_is_join_node):
  # print("EVALUATING current: ",current)
  assert type(child) == int, f"expect int type for child but type is: {type(child)}"

  # print(f"Correcting the unnice node {current} with child: {child}")
  current_values = graph.nodes[current]["values"]
  child_values = graph.nodes[child]["values"]
  current_values_set = set(current_values)
  child_values_set = set(child_values)


  ### IDENTICAL CASE
  if current_is_join_node == False and current_values_set == child_values_set:
    ### IDENTICAL CASE
    # OK just delete one of the nodes? wtf why did I insert a bridging node previously?
    # new_node_id = handle_identical(graph,current,child,current_values_set,child_values_set)
    child_neighbors = list(graph.neighbors(child))
    ### Handling the case where a non-join node parent and a child have identical bags, BUT
    ### where the child node itself is a join node. In this case, it is easier to remove the parent node.
    if len(child_neighbors) == 3:
      # print(f"Child is a join node. Must remove the parent instead")
      parent_of_current = list(graph.neighbors(current))
      ### If this is the case, we have two join nodes in a row. Throwing exception here
      if len(parent_of_current) == 3:
        raise Exception("nononononoooo, two join nodes in a row??")
      parent_of_current.remove(child)
      parent_of_current = parent_of_current[0]
      # print(f"Removing current: {current} which has neigbors: {list(graph.neighbors(current))}")
      graph.remove_node(current)
      # print(f"Adding edge between parent: {parent_of_current} and child: {child}")
      graph.add_edge(parent_of_current,child)
      make_decomposition_nice(graph,child,parent_of_current)
    else:
      # print(f"Current: {current} child: {child} neighbors of child: {child_of_child}. Current_values_set: {current_values_set} and child: {child_values_set}")
      child_of_child = child_neighbors
      child_of_child.remove(current)
      child_of_child = child_of_child[0]
      # print(f"Should be child of child: {child_of_child}")
      # print(f"REMOVING THE NODE: {child} which has neighbors: {list(graph.neighbors(child))}")
      graph.remove_node(child)
      # print(f"Adding edge between {current} and {child_of_child}")
      graph.add_edge(current,child_of_child)
      # current_parent = list(graph.neighbors(current))
      # current_parent.remove(child_of_child)
      # print(f"IDENTICA CASE. Callings recurse for current with parent: {current, current_parent}")
      correct_unnice_node(graph,current,child_of_child,False)
      # make_decomposition_nice(graph, current, child_of_child)
  elif len(current_values_set-child_values_set) > 0 and len(child_values_set-current_values_set) > 0:
    ### AMBIVALENT CASE
    # print("The current values contain  at least one element not present in child values, and vice versa. This means that the current both forgets and introduces at the same time")
    # print(f"Handling ambivalent case. calling handle eager introducer with current: {current} child: {child}")
    new_node_id = handle_eager_introducer(graph,current,child,current_values_set,child_values_set)
    make_decomposition_nice(graph, new_node_id, current)
  elif len(current_values_set-child_values_set) > 1:
    ### EAGER INTRODUCER CASE
    # print("current values contains 2 or more elements more than child value does. Introduces more than 1 element ")
    # print(f"calling handle eager introducer from introduce case. current, child: {current, child}")
    new_node_id = handle_eager_introducer(graph,current,child,current_values_set,child_values_set)
    make_decomposition_nice(graph, new_node_id, current)
  elif len(child_values_set-current_values_set) > 1:
    ### AMNESIAC PARENT CASE
    # print(f"Current: {current} is amnesiac")
    # print("Child value contains 2 or more elements more than parent. must forget too much at once")
    new_node_id = handle_amnesiac_node(graph,current,child,current_values_set,child_values_set)
    make_decomposition_nice(graph, new_node_id, current)
  else:
    # print(f"Current node {current} has no violation with child {child}. Recursing on child again")
    make_decomposition_nice(graph,child,current)



# Recursive (well bi-directionally recursive) function that goes through the decomposition and calls to fix nodes
def make_decomposition_nice(graph, current, parent):
  # Find the children of this current current node by removing the parent from
  # the list of its neighbors
  children = list(graph.neighbors(current))
  try:
    children.remove(parent)
  except ValueError:
    # print(f"Current current: {current} has no parent, is probably current of entire tree")
    pass
  # print(f"In recursion. current: {current},  children: {children}")
  # print(f"Making decomp nice on current: {current}")
  # If this node has no children, it must be a current
  if len(children) == 0:
    # print(f"len of children of this node: {current} is 0, must be a leaf. returning")
    return
  # Otherwise, if this node has 2 children, it must be a join node
  elif len(children) > 1:
    # print(f"current node {current} has more than 1 child and must be a join node")
    assert len(children) == 2, f"Expected children to be of length 2 but is: {len(children)}"
    # In the case of a join node, we branch out to each of the children
    # print(f"Branching out to the two children of the current: {current} whchi is a join node")
    correct_unnice_node(graph, current, children[0], True)
    correct_unnice_node(graph, current, children[1], True)
  else:
    # Otherwise, the current current has just one child and we continue
    assert len(children) == 1, f"Expected children to be of length 1 but is: {len(children)}"
    # print(f"Calling correct unnice from current. Only has 1 child: {current}")
    correct_unnice_node(graph, current, children[0], False)


In [144]:
def construct_nice_decomposition(input_graph):
  decomposition, root = construct_junction_and_add_leaves(input_graph)
  handle_promiscuous_nodes2(decomposition,root,None)
  handle_join_nodes(decomposition,root,None)
  make_decomposition_nice(decomposition, root, None)
  # fix_useless_introducers(decomposition,root,None,None)
  return decomposition, root

In [145]:
def count_matchings_simple(graph):
  edge_list = list(graph.edges)
  matchings = 0
  for r in range(1+math.floor(graph.number_of_nodes()/2)):
    r_combinations = itertools.combinations(edge_list,r)
    for comb in r_combinations:
      if (nx.is_matching(graph,comb) == True):
        matchings += 1
  return matchings

In [146]:
def powerset(iterable,min_inclusive,max_exclusive):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(min_inclusive, max_exclusive))

# This should not necessarily be commented out, just testing.

In [80]:
# def count_matchings_forcibly_saturate_vertex(input_graph, vertex_to_saturate):
#   neighbors = list(input_graph.neighbors(vertex_to_saturate))
#   # print()
#   # print()
#   # print("IN COUNT FORIBLY SAT")
#   # print("neighbors: ",neighbors)
#   matchings = []
#   for saturation_endpoint in neighbors:
#     graph = input_graph.copy()
#     this_edge = ((vertex_to_saturate,saturation_endpoint),(saturation_endpoint,vertex_to_saturate))

#     other_neighbors = neighbors.copy()
#     other_neighbors.remove(saturation_endpoint)
#     for other_neighbor in other_neighbors:
#       graph.remove_edge(vertex_to_saturate,other_neighbor)

#     # print(f"After removing neighbors from {vertex_to_saturate}, nodes,edges:")
#     # print(graph.nodes)
#     # print(graph.edges)


#     # print("this edge is: ",this_edge)
#     all_other_edges = list(graph.edges())
#     true_edge = None
#     try:
#       all_other_edges.remove(this_edge[0])
#       true_edge = this_edge[0]
#     except:
#       all_other_edges.remove(this_edge[1])
#       true_edge = this_edge[1]

#     powerset_of_other_edges = powerset(all_other_edges,1,round(len(graph.nodes)/2))

#     count = 1
#     for subset_of_other_edges in list(powerset_of_other_edges):
#       # print("These matching? :", true_edge, subset_of_other_edges, nx.is_matching(graph,set({true_edge,*subset_of_other_edges})))
#       if nx.is_matching(graph,set({true_edge, *subset_of_other_edges})):
#         count += 1
#     matchings.append(count)

#   print("Before returning. matchigns is: ",matchings)
#   return sum(matchings)

# NOT WORKING FULLY THIS ONE I BELIEVE

In [81]:
def discriminately_test_edges(graph, edges_to_saturate, edges_to_check,):
    def inner(graph,edges_to_saturate, remainder,accumulator):

        print()
        print("in start, total: ")
        print(f"remainder: {remainder}, accumulator: {accumulator}")
        total_res = 0
        # if nx.is_matching(graph,edges_to_saturate+accumulator) == False:
        print(f"Calling is matching on graph nodes, edges: {graph.nodes} and {graph.edges} with this list: {[*edges_to_saturate,*accumulator]}")
        is_matching = nx.is_matching(graph,[*edges_to_saturate,*accumulator])
        print("Was it a match? ",is_matching)
        if is_matching == False:
            print(f"NOT A MATCH: {accumulator} in graph.nodes, edges: {graph.nodes} and {graph.edges}")
            return total_res
        # else:
            # return 1
            # total_res += 1
        for i in range(len(remainder)):
            cut_element = remainder[i]
            cut_remainder = remainder[(i+1):]
            print("cu remainder: ",cut_remainder)
            res = inner(graph, edges_to_saturate, cut_remainder, accumulator + [cut_element])
            total_res += res
            print("yaya")
            if res == 0:
                continue

        if len(accumulator) > 0:
            print(f"At end. returning total res: {total_res} plus 1 for accumulator: {accumulator}")
            return total_res+1
        else:
            print(f"At TOTAL end. returning total res: {total_res} for accumulator: {accumulator}")
            return total_res
    # inner(l[1:],l[0:1])
    big_res = inner(graph, edges_to_saturate, edges_to_check,[])
    # print("bigres: ",big_res)
    return big_res

# delete ?


In [82]:
def discriminately_test_edges(graph, edges_to_saturate, edges_to_check,):
    def inner(graph,edges_to_saturate, remainder,accumulator,value):

        print()
        print("in start, total: ")
        print(f"remainder: {remainder}, accumulator: {accumulator}")
        # if nx.is_matching(graph,edges_to_saturate+accumulator) == False:
        print(f"Calling is matching on graph nodes, edges: {graph.nodes} and {graph.edges} with this list: {[*edges_to_saturate,*accumulator]}")
        is_matching = nx.is_matching(graph,[*edges_to_saturate,*accumulator])
        print("Was it a match? ",is_matching)


        if is_matching == False:
            print(f"NOT A MATCH: {accumulator} in graph.nodes, edges: {graph.nodes} and {graph.edges}")
            return 0
        else:
            # return value + 1
            print(f"incrementing value from: {value} to {value+1}")
            value += 1
        for i in range(len(remainder)):
            cut_element = remainder[i]
            cut_remainder = remainder[(i+1):]
            print("cu remainder: ",cut_remainder)
            print(f"CALLIGN ERCURIVELy where value is: {value}")
            res = inner(graph, edges_to_saturate, cut_remainder, accumulator + [cut_element],value)
            value += res
            if res == 0:
                return value
            # if res == 0:
            else:
                return value+1
            #     break
            # value = res
        if len(accumulator) > 0:
            print(f"At end. returning total res: {value} plus 1 for accumulator: {accumulator}")
            return value +1
        else:
            print(f"At TOTAL end. returning total res: {value} for accumulator: {accumulator}")
            return value
    # inner(l[1:],l[0:1])
    big_res = inner(graph, edges_to_saturate, edges_to_check,[],0)
    # print("bigres: ",big_res)
    return big_res

In [83]:
# def outer(list):
def recurse(l,accumulator,counter):
	print("COUNTER: ",counter)
	print(f"l: {l} acc: {accumulator}")
	# print("len(l) is: ",len(l))
	if sum(accumulator) < 2:
		return 0
	# if len(l) == 0:
	# 	print("returning 1")
	# 	return 1
	# for element in l:
	result = 0
	for i in range(0,len(l)):
		# print("i is:",i)
		# print("element:",l[i])
		# if True:
		head = l[i]
		print(f"Calling recurse on {l} and {[*accumulator,head]}")
		# print(f"Calling recurse on {l} and {accumulator+[head]}")
		result += recurse(l[i+1:],[*accumulator,head],counter)
	if len(accumulator) > 0:
		return result + 1
	else:
		return result
		# results.append(recurse(l[i+1:],[*accumulator,head],counter+1))
	return counter

# original dirty

In [84]:
def get_matching_identities(graph,separator_vertices):
  edge_list = list(graph.edges)

  n_nodes = graph.number_of_nodes()
  ##### [matching1 : "10", matching2: "00", matching: "00"]
  # bin_str = []
  ##### map(str, int) = map(identity, total number of occurrences) 
  bin_map = {}
  # matchings = []
  for r in range(1+math.floor(n_nodes/2)):
    r_combinations = itertools.combinations(edge_list,r)
    for comb in r_combinations:
      if (nx.is_matching(graph,comb)):
        # current_str = ["0" for x in range(max(graph.nodes)+1)]
        current_str_list = ["0" for x in range(len(separator_vertices))]
        ###### NOTE: Now no longer read from the right. Inherent ordering doesn't matter. Just need to be consistent with indexing
        # current_str = "0"*len(separator_vertices)
        for val in comb:
          for i in range(len(separator_vertices)):
            if (separator_vertices[i] in val):
              current_str_list[i] = "1"
              # current_str[-(i+1)] = "1"
          # for sep in separator_vertices:
          #   if sep in val:
          #     current_str[-(sep+1)] = "1"
            # else:
        # matchings.append(comb)
        # bin_str.append(current_str_list)
        ##### if map contains current_str: increment by 1, otherwise add current_str with value = 1
        current_str = "".join(current_str_list)
        try:
          bin_map[current_str] += 1
        except KeyError:
          bin_map[current_str] = 1
  return bin_map
  # return bin_str
  # return (matchings,bin_str)



def count_matchings_separators(graph, separator_vertices,verbose):
  # Construct the disconnected graph by removing the separator vertices from the original graph (and storing a copy)
  init1 = time.time()
  G_disconnected = graph.copy()
  for node in separator_vertices:
    G_disconnected.remove_node(node)

  # Find the connected subgraphs of this disconnected parent graph, store them in a list
  # subgraphs = [G_disconnected.subgraph(component) for component in nx.connected_components(G_disconnected)]
  connected_components = nx.connected_components(G_disconnected)
  # for component in connected_components:
  #   print(component)
  subgraphs = [graph.subgraph(comp) for comp in connected_components]
  n_subgraphs = len(subgraphs)
  if n_subgraphs < 2:
    raise Exception("These separator vertices only partitioned the graph into 1 subgraph")
  # print(n_subgraphs)
  # Append the separator vertices to each of these connected subgraphs
  #####
  ##### Less-ghetto approach: delete edges between separator vertices in all 
  ##### except 1 "recombined subgraph" (but keep separator vertices in all subgraphs)
  ##### 
  #####
  recombined = [graph.subgraph(list(subgraph) + separator_vertices) for subgraph in subgraphs]

  init2 = time.time()
  # This is ghetto for now - fix it!
  # I realized a bug that I had in my implementation before: I was counting
  # matchings that include edges between separator vertices multiple times since
  # those edges were contained in both subgraphs after I reintroduced the separator
  # vertices to each of them.
  sept1 = time.time()
  edges_between_separator_vertices = []
  for separator_vertex_pair in itertools.combinations(separator_vertices,2):
    if graph.has_edge(separator_vertex_pair[0],separator_vertex_pair[1]):
      edges_between_separator_vertices.append((separator_vertex_pair[0],separator_vertex_pair[1]))

  # Iterating from 1 since we save the first entry in recombined to account for edges
  # between separator vertices just once
  # recombined_corrected = []
  # for i in range(1,len(recombined)):
  #   recombined_cop = recombined[i].copy()
  #   for edge in edges_between_separator_vertices:
  #     print("Removing edge: ",edge)
  #     recombined_cop.remove_edge(edge[0],edge[1])
  #   recombined_corrected.append(recombined_cop)
  recombined_purged = [recombined[0]]
  for i in range(1,len(recombined)):
    recombined_cop = recombined[i].copy()
    for edge in edges_between_separator_vertices:
      print("Removing edge: ",edge)
      recombined_cop.remove_edge(edge[0],edge[1])
    recombined_purged.append(recombined_cop)
  
  sept2 = time.time()
  
  # Compute all matchings for each of these subgraphs. Matchings are stored
  # by which separator vertices they saturate. Saturation of a separator vertex
  # is encoded in a binary string. The length of the string is equal to the 
  # number of separator vertices where each position in the binary string (each bit)
  # corresponds to one separator vertex. A 1 in a bit indicates
  # that the matching saturates that separator vertex.
  mid1 = time.time()
  # matching_identities_in_each_subgraph = [get_matching_identities(connected_component,separator_vertices) for connected_component in recombined_purged]
  matching_identities_in_each_subgraph = []
  for cc in recombined_purged:
    if verbose == True:
      print(f"V,E for current connected component: {cc.number_of_nodes()}, {cc.number_of_edges()}")
      cc_start = time.time()
      matching_identities_in_each_subgraph.append(get_matching_identities(cc,separator_vertices))
      cc_stop = time.time()
      print(f"Time spent getting matching identities for connected component: {round(cc_stop-cc_start,4)}")
    else:
      matching_identities_in_each_subgraph.append(get_matching_identities(cc,separator_vertices))


  # matching_identities_in_each_subgraph = [get_matching_identities(connected_component,separator_vertices) for connected_component in recombined_purged]
  # print(matching_identities_in_each_subgraph)
  mid2 = time.time()
  vt1 = time.time()
  valid_matchings = 0
  # "matchings_in_each_subgraph" is a list of lists. The parent list has length
  # equal to the number of disjoint, connected components in the separator
  # vertex-disconnected parent graph. Each of these sublists contains all 
  # the matchings found in the
  # subgraphs enriched with separator vertices.

  ##### Old method:
  # for combination in product(*matchings_in_each_subgraph):
  #   valid = 1
  #   for comb in combinations(combination,2):
  #     if int("".join(comb[0]),2) & int("".join(comb[1]),2) != 0:
  #       valid = 0
  #   valid_matchings += valid
  ##### update to use map, e.g.:
  ##### map1 ("00":100, "01":38, "10":1)
  ##### map2 ("00":10, "01":28, "10":38)
  ##### map1.get("01")*map2.get("00")+map1.get("01")*map1.get("10")
  # possible_identity_strings = list(matching_identities_in_each_subgraph[0].keys())
  identity_strings_for_each_subgraph = [list(x.keys()) for x in matching_identities_in_each_subgraph]
  # print("identity_strings_for_each_subgraph: ",identity_strings_for_each_subgraph)

  # for single_identity_product in product(*[identity_strings_for_each_subgraph for x in range(n_subgraphs)]):
  for single_identity_product in itertools.product(*identity_strings_for_each_subgraph):
    # print("single id product: ", single_identity_product)
    is_valid = -1
    is_valid = int(single_identity_product[0],2) & int(single_identity_product[1],2)
    # print("Is valid? 0 is valid ", is_valid)
    # valid if is_valid is still = 0
    if is_valid == 0:
      subgraph_matchings_counts = []
      # subgraph_matchings_product = 0
      for i in range(n_subgraphs):
        subgraph_matchings_counts.append(matching_identities_in_each_subgraph[i][single_identity_product[i]])
      # print("Adding this: ", functools.reduce(lambda el, acc:  el * acc, subgraph_matchings_counts)) 
      valid_matchings += functools.reduce(lambda el, acc:  el * acc, subgraph_matchings_counts)



  vt2 = time.time()
  if (verbose == True):
    print(f"{round(init2-init1,4)}: time spent on init")
    print(f"{round(sept2-sept1,4)} time spent on ghetto edge separation thing")
    print(f"{round(mid2-mid1,4)}: time spent on getting matching identities for the whole lot")
    print(f"{round(vt2-vt1,4)}: Time spent on validation")
  return valid_matchings

In [85]:
def count_matchings_vertex_included_multiple(input_graph, vertices_to_saturate:list):
    if vertices_to_saturate != list:
        vertices_to_saturate = list(vertices_to_saturate)
    assert type(vertices_to_saturate) == list
    vertices_to_consider = copy.deepcopy(vertices_to_saturate)
    # print("Vertices to consider after init: ",vertices_to_consider)
    assert vertices_to_consider == vertices_to_saturate
    # print(f"Vertices to consider: {vertices_to_consider} in graph nodes: {input_graph.nodes}",)

    # vertices_to_saturate_which_are_neighbors = []
    # # print("Vertices to consider2",vertices_to_consider)
    # for i in range(len(vertices_to_saturate)):
    #   # print("Vertices to consider just after i iterator",vertices_to_consider)
    #   for j in range(i+1,len(vertices_to_saturate)):
    #     # print("Vertices to consider just after j iterator",vertices_to_consider)
    #     v1 = vertices_to_saturate[i]
    #     v2 = vertices_to_saturate[j]
    #     if input_graph.has_edge(v1,v2):
    #       # print(f"These are adjacent: {v1}, {v2}")
    #       vertices_to_saturate_which_are_neighbors.append((v1,v2))
    #       try:
    #         vertices_to_consider.remove(v1)
    #         vertices_to_consider.remove(v2)
    #       except:
    #         # TODO: Verify the veracity of this. If we are not able to remove, it must mean that we have already removed.
    #         # This example can occur if we wish to saturate the vertices {1,2,3} in the graph but that we have edges
    #         # (1,2), (2,3).
    #         return 0
    #       # except:
    #       #   print(f"exception while removing v  {v1} from the list: {vertices_to_consider}")
    #       #   print("input graph nodes, edges:")
    #       #   print(input_graph.nodes)
    #       #   print(input_graph.edges)
    #       # try:
    #       #   vertices_to_consider.remove(v2)
    #       # except:
    #       #   print(f"exception while removing v2 {v2} from the list: {vertices_to_consider}")
    # print(f"Vertices to saturate which are neighbors: {vertices_to_saturate_which_are_neighbors}")
    # List of lists. [[v1.neighbors],[v2.neighbors],[..],...]
    # Cartesian product of v1.neighbors, v2.neighbors, ...
    # So each entry in this variable representss one way of saturating the vertices
    # which must be saturated. Thereby they are fixed, and we then count no. of matchings
    # in the subgraph that results
    vertices_to_saturate_neighbors = [list(input_graph.neighbors(x)) for x in vertices_to_consider]
    # Ways that these vertices to saturate may be saturated
    saturation_possibilities = list(itertools.product(*vertices_to_saturate_neighbors))
    # print("sat possibilities: ", saturation_possibilities)


    total_matchings = 0
    total_matchings_discriminately = 0
    for saturation_direction_instance in saturation_possibilities:
        graph = input_graph.copy()
        # print(f"After graph copy, nodes: {graph.nodes}, edges: {graph.edges}")
        # if len(vertices_to_saturate_which_are_neighbors) > 0:
        #   for pair in vertices_to_saturate_which_are_neighbors:
        #     for vertex in pair:
        #       for neighbor in list(graph.neighbors(vertex)):
        #         if neighbor not in pair:
        #           print(f"Removing edge between {vertex} and {neighbor}")
        #           try:
        #             graph.remove_edge(vertex,neighbor)
        #           except:
        #             print("Except")
        #             print("Except")
        #             print("Except")
        #             print("Except")
        # if len(saturation_possibilities) == 0:
        #   print("We have no saturation possibilities.")

        neighbor_vertices_taken_up_for_matching = dict()
        # print(f"Evaluating instance: {saturation_direction_instance}")
        if len(saturation_direction_instance) > len(set(saturation_direction_instance)):
            # print("There is a duplicate saturation direction vertex. This must be impossible. Continuing to next iteration")
            continue
        assert len(saturation_direction_instance) == len(vertices_to_consider)
        saturation_direction_instance_is_valid = True
        for i in range(len(saturation_direction_instance)):
            # "home vertex" is the vertex of vertices_to_consider that is currently
            # under investigation. its partner is the neighbor to which it is
            # matched in the current instance.
            if saturation_direction_instance_is_valid == False:
                # print("Breaking, saturation direction instance is invalid")
                break
            home_vertex = vertices_to_consider[i]
            partner_vertex = saturation_direction_instance[i]
            # print(f"Home vertex: {home_vertex}, partner_vertex: {partner_vertex}")
            if graph.has_edge(home_vertex, partner_vertex) == False:
                # print(f"ERROR: graph currently has no edge between home vertex {home_vertex} and partner {partner_vertex}, edges: {graph.edges}")
                saturation_direction_instance_is_valid = False
                continue
            partner_vertex_neighbors_to_remove = list(graph.neighbors(partner_vertex))
            partner_vertex_neighbors_to_remove.remove(home_vertex)
            # print("Partner vertex neighbors to remove: ",partner_vertex_neighbors_to_remove)
            # if can_remove_partner_vertex_neighbors_in_instance(partner_vertex_neighbors_to_remove,saturation_direction_instance) == False:
            #   saturation_direction_instance_is_valid = False
            #   continue
            # for neighbor_to_remove in partner_vertex_neighbors_to_remove:
            #   if neighbor_to_remove in saturation_direction_instance:
            #     print(f"Heyeyey this won't go. We are trying to remove {neighbor_to_remove} but it is part of the saturtion direction instnace: {saturation_direction_instance}")
            #     break
            # If the designated partner of this home vertex is already taken up for matching by another
            # vertex, the current saturation direction instance cannot fulfill the constraints, and we must return 0
            if partner_vertex in neighbor_vertices_taken_up_for_matching:
                # print(f"Ey problem, the designated partner {partner_vertex} of the home vertex {home_vertex} is already taken up by the home vertex {neighbor_vertices_taken_up_for_matching[partner_vertex]}")
                # (return 0)
                # TODO: Fix this, should it return 0 globally?
                raise Exception("eyeyeyeyeyey")
                return 0
            else:
                neighbor_vertices_taken_up_for_matching[partner_vertex] = home_vertex
            # Iterate over neighbors of the home vertex and remove all edges from
            # home vertex to its neighbors (except its designated partner)
            # print("Home vertex is: ",home_vertex)
            home_vertex_neighbors = list(graph.neighbors(home_vertex))
            # print(f"Attempting to remove {partner_vertex} from {home_vertex_neighbors}")
            home_vertex_neighbors.remove(partner_vertex)
            for home_neighbor_to_unedge in home_vertex_neighbors:
                # print(f"Removing edge between {home_vertex} and {home_neighbor_to_unedge}")
                graph.remove_edge(home_vertex,home_neighbor_to_unedge)
            # for neighbor_of_home_vertex in list(graph.neighbors(home_vertex)):
            #   if 
            for edge_to_remove in partner_vertex_neighbors_to_remove:
                # print(f"Removing edge between {partner_vertex} and {edge_to_remove}")
                graph.remove_edge(partner_vertex,edge_to_remove)
        if saturation_direction_instance_is_valid == False:
            continue
        edges_except_constraints = list(graph.edges)
        # print("EEC immediately after init: ", edges_except_constraints)
        # This is quite ghetto but is just to avoid creating a copy of the entire graph for the sole purpose of removing
        # these edges.
        for i in range(len(vertices_to_consider)):
        #   if graph.has_edge(vertices_to_consider[i],saturation_direction_instance[i]):   
            # print(f"Trying to remove: {vertices_to_consider[i],saturation_direction_instance[i]} from {edges_except_constraints}")
            if len(edges_except_constraints) > 0:
                try:
                    edges_except_constraints.remove((vertices_to_consider[i],saturation_direction_instance[i]))
                except:
                    pass
                try:
                    edges_except_constraints.remove((saturation_direction_instance[i],vertices_to_consider[i]))
                    # edges_except_constraints.remove((vertices_to_consider[i],saturation_direction_instance[i]))
                except:
                    pass
        # for pair in vertices_to_saturate_which_are_neighbors:
        #   try:
        #     edges_except_constraints.remove(pair)
        #   except ValueError:
        #     print(f"pair not in list: ", pair)
        #     print("the list: ", edges_except_constraints)
        #     edges_except_constraints.remove((pair[1],pair[0]))
        # print("EEC after: ",edges_except_constraints)

        ps = list(powerset(edges_except_constraints,1,1+math.floor((graph.number_of_nodes()/2))))
        # itertools.combinations(s, r) for r in range(min_inclusive, max_exclusive)
        # print(f"POwerset contains: {ps}")
        # print(f"About to construct saturation edges in instance. Sat direction instance: {saturation_direction_instance} vertices to consider: {vertices_to_consider}")
        saturation_edges_in_instance = set()
        for i in range((len(vertices_to_consider))):
            saturation_edges_in_instance.add(tuple(sorted((saturation_direction_instance[i],vertices_to_consider[i]))))
        # print(f"saturation edges in instance: {saturation_edges_in_instance} and edges wo constr {edges_except_constraints}")
        # eh_mx = discriminately_test_edges(graph, list(saturation_edges_in_instance),edges_except_constraints)
        # total_matchings_discriminately += eh_mx
        # print("ehmx:",eh_mx)
        # saturation_edges_in_instance = [(saturation_direction_instance[i],vertices_to_consider[i]) for i in range(len(vertices_to_consider))]
        # print(saturation_edges_in_instance)
        # print(f"Setting local matchings to 1 for these edges: {saturation_edges_in_instance}")
        # print(f"Setting local matchings to 1 for these edges: {saturation_edges_in_instance}")
        # print(f"Setting local matchings to 1 for these edges: {saturation_edges_in_instance}")
        local_matchings = 1
        # print("Hello?")
        # invalid_subsets = set()
        for element_in_powerset in ps:
            # print("is it me?")
            edges_to_evaluate = [*saturation_edges_in_instance,*element_in_powerset]
            # print(f"Evaluating is matching?: {edges_to_evaluate}")
            is_matching = nx.is_matching(graph, edges_to_evaluate)
            if is_matching:
                # print(f"This is matching: {edges_to_evaluate} in the graph w edges {graph.edges}")
                # print(f"This is matching: {edges_to_evaluate} in the graph w edges {graph.edges}")
                # print(f"This is matching: {edges_to_evaluate} in the graph w edges {graph.edges}")
                local_matchings += 1
            # else:
            #     invalid_subsets.add(edges_to_evaluate)
        # print("Powerset found local matchings: ",local_matchings)
        total_matchings += local_matchings
    #   print("LOCAL MATCHINGS COUNTER: ",local_matchings)
    #   print("TOTAL MATCHINGS COUNTER: ",total_matchings)
    # print("Returning: ",total_matchings)
    # print("tmd: ",total_matchings_discriminately)
    return total_matchings

In [86]:
def count_matchings_forcibly_saturate_and_unsaturate(input_graph, vertices_to_saturate: list, vertices_to_unsaturate: list):
    graph = input_graph.copy()
    if type(vertices_to_unsaturate) != list:
        vertices_to_unsaturate = list(vertices_to_unsaturate)
    if len(vertices_to_unsaturate) > 0:
        # print(f"removing nodes from: {vertices_to_unsaturate}")
        graph.remove_nodes_from(vertices_to_unsaturate)
    # print(f"Calling count matchings vertex included multiple with g.nodes: {graph.nodes} and v to sat: {vertices_to_saturate}")
    return count_matchings_vertex_included_multiple(graph,vertices_to_saturate)

In [87]:
pcm_decomp, pcm_decomp_root = construct_nice_decomposition(pcm)

Making decomp nice on current: 16
Calling correct unnice from current. Only has 1 child: 16
Correcting the unnice node 16 with child: 2
Current: 16 is amnesiac
Making decomp nice on current: 21
Calling correct unnice from current. Only has 1 child: 21
Correcting the unnice node 21 with child: 2
Current node 21 has no violation with child 2. Recursing on child again
Making decomp nice on current: 2
Calling correct unnice from current. Only has 1 child: 2
Correcting the unnice node 2 with child: 13
The current values contain  at least one element not present in child values, and vice versa. This means that the current both forgets and introduces at the same time
Handling ambivalent case. calling handle eager introducer with current: 2 child: 13
Calling insert bridgin node with root: 2 and child: 13 with graph edges: [(0, 9), (0, 10), (1, 11), (1, 12), (2, 13), (2, 21), (3, 17), (3, 19), (4, 14), (4, 15), (5, 12), (5, 18), (6, 9), (6, 14), (7, 10), (7, 20), (8, 11), (8, 15), (13, 19), (13

In [88]:
halicin = read_smiles("C1=C(SC(=N1)SC2=NN=C(S2)N)[N+](=O)[O-]")
# halicin_graph, halicin_root = construct_junction_and_add_leaves(halicin)
# handle_promiscuous_nodes2(halicin_graph,halicin_root,None)
# handle_join_nodes(halicin_graph,halicin_root,None)
# print(halicin_graph.nodes(data=True))
# print(halicin_graph.edges)
# ENTRY(halicin_graph,halicin_root)

# Original, dirty version (dirty w regards to print statements and commented-out lines)

In [89]:
# state_results = {}

# def count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition, current, parent):
#   children = list(decomposition.neighbors(current))
#   try:
#     children.remove(parent)
#   except ValueError:
#     print("Current element has no parent, must be root")
#   print(f"Current node: {current} has these children after removal of parent: {children}")
#   while len(children) > 0:
#     if len(children) == 2:
#       assert len(children) == 2, f"Expected 2 children, got {len(children)}"
#       assert type(children) == list, f"TYpe of children should be list, it is {children}"
#       print("calling left")
#       child_states_left, vertices_encountered_left_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[0],current)
#       print("calling right")
#       child_states_right, vertices_encountered_right_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[1],current)
#       print("AT JOIN NODE")
#       print("AT JOIN NODE")
#       print("AT JOIN NODE")
#       print(f"child states left: ",child_states_left)
#       print(f"child states right: ",child_states_right)
#       if vertices_encountered_left_set.issubset(vertices_encountered_right_set) or vertices_encountered_right_set.issubset(vertices_encountered_left_set):
#         print(f"Ey whats going on, one subgraph is a susbet of the other? {vertices_encountered_left_set} and {vertices_encountered_right_set}")
#         if len(vertices_encountered_left_set) > len(vertices_encountered_right_set):
#           return child_states_left, vertices_encountered_left_set
#         else:
#           return child_states_right, vertices_encountered_right_set
#         # return max(sum_left,sum_right), vertices_encountered_left_set.union(vertices_encountered_right_set)  
#       # else:    
#       print("At join node. Current: ",current)
#       vertices_encountered_left = list(vertices_encountered_left_set)
#       vertices_encountered_right = list(vertices_encountered_right_set)
#       # print("VE_left: ",vertices_encountered_left)
#       # print("VE_right: ",vertices_encountered_right)
#       vertices_encountered_left.sort()
#       vertices_encountered_right.sort()
#       # induced_subgraph_left = graph.subgraph(vertices_encountered_left)
#       # induced_subgraph_right = graph.subgraph(vertices_encountered_right)
#       current_bag = decomposition.nodes[current]["values"]
#       left_child_bag = decomposition.nodes[children[0]]["values"]
#       right_child_bag = decomposition.nodes[children[1]]["values"]
#       current_bag.sort()
#       left_child_bag.sort()
#       right_child_bag.sort()
#       indices_of_current_bag_elements_in_left_bag = [left_child_bag.index(x) for x in current_bag]
#       indices_of_current_bag_elements_in_right_bag = [right_child_bag.index(x) for x in current_bag]
#       powerset_of_current_bag = list(powerset(current_bag,0,len(current_bag)+1))
#       # print(f"About to branch out left and right. current_bag: {current_bag}, induced subgraph nodes left, right: {induced_subgraph_left.nodes} and {induced_subgraph_right.nodes}")
#       # running_sum = 0


#       # outer_sum
#       new_states = {}
#       state_sum_accumulator = 0
#       # Iterates over all possible states at the join node, e.g. "000", "001", "010", "011", etc
#       for state_string_tuple in list(itertools.product("01",repeat=len(current_bag))):
#         state_string = "".join(state_string_tuple)
#         print()
#         print("STATE STRING: ",state_string)
#         # Find the indices in the string which contain a 1
#         # e.g. "010" -> [1],          "011" -> [1,2]
#         indices_of_ones = []
#         for i in range(len(state_string)):
#             if state_string[i] == "1":
#                 indices_of_ones.append(i)
#         # indices_of_ones = [state_string[int(x] for x in state_string if x == "1"]
#         # print("INDIECS OF ONS: ",indices_of_ones)
#         # Compute ways to distribute these ones to the two subgraphs,
#         # e.g. [1] -> (([1],),(,[1])     [1,2] -> (([1,2],),([1],[2]),([2],[1]),(,[1,2]))
#         # state_result = 0
#         # running_sum_left = 0
#         # running_sum_right = 0
#         state_inner_accumulator = 0
#         for indices_of_ones_assigned_to_left in list(powerset(indices_of_ones,0,len(state_string)+1)):
#             indices_of_ones_assigned_to_right = list(set(indices_of_ones)-set(indices_of_ones_assigned_to_left))
#             # print(f"indices assigned left: {indices_of_ones_assigned_to_left} and right: {indices_of_ones_assigned_to_right}")
#             # Now, for e.g. the state string "011" and the index example [1,2] -> ([1],[2]) 
#             # this can be interpreted as "match index 1 of the state string in the left subgraph, match index 2 of the state string in the right subgraph"
#             # I must now fill the "rest" of the state string indices with 0's in order to forcibly unsaturate them
#             # then, I will obtain these signatures: subgraph1: "010" subgraph2: "001"
#             print(f"indiced of ones assigned to left: {indices_of_ones_assigned_to_left} and right: {indices_of_ones_assigned_to_right}")
#             state_string_left = ["0" for x in range(len(state_string))]
#             for index_of_one_to_left in indices_of_ones_assigned_to_left:
#                 state_string_left[int(index_of_one_to_left)] = "1"
                
#             state_string_right = ["0" for x in range(len(state_string))]
#             # print(f"state string right: {state_string_right}")
#             for index_of_one_to_right in indices_of_ones_assigned_to_right:
#                 state_string_right[int(index_of_one_to_right)] = "1"
#             # state_string_left should in the example now contain "010" and the right variable contain "001"
#             print(f"statestringleft: {state_string_left} and statesright: {state_string_right}")

#             # graph_string_left = ["0" for x in range(len(v_enc_left))]
#             # graph_string_right = ["0" for x in range(len(v_enc_right))]

#             # indices_of_ones_in_left_graph = []
#             # indices_of_ones_in_right_graph = []
#             # for i in range(len(state_string_left)):
#             #     if state_string_left[i] == "1":
#             #         indices_of_ones_in_left_graph.append(vertices_encountered_left.index(current_bag[i]))
#             #         # graph_string_left[v_enc_left.index(join_node_bag[i])] = "1"
#             # for i in range(len(state_string_right)):
#             #     if state_string_right[i] == "1":
#             #         indices_of_ones_in_right_graph.append(vertices_encountered_right.index(current_bag[i]))
#             #         # graph_string_right[v_enc_right.index(current_bag[i])] = "1"
#             print(f"Now ready to go, I think. join bag: {current_bag} bags left, right: {vertices_encountered_left} and {vertices_encountered_right}")
#             # print(f"And graph string left right: {graph_string_left} and {graph_string_right}")
#             print()
#             print(f"state string left graph: {state_string_left}")
#             print(f"state string right graph: {state_string_right}")
#             print()


#             left_value = child_states_left["".join(state_string_left)]
#             right_value = child_states_right["".join(state_string_right)]
#             combined_value = left_value*right_value
#             state_inner_accumulator += combined_value
#             print("left value: ",left_value)
#             print("right value: ",right_value)



#             # # Now iterate over all of the stored states from each child. Find those which have the same matching signature
#             # # as the state that we are investigating now. If 
#             # inner_sum_left = 0
#             # for left_child in child_states_left:
#             #   print("left child is: ",left_child)
#             #   print(f"indices of one in left graph: {state_string_left}")
#             #   for i in range(len(state_string_left)):
#             #     no_violation = True
#             #     if left_child[int(state_string_left[i])] != "1":
#             #       no_violation = False
#             #   if no_violation:
#             #     # running_sum_left += child_states_left[left_child[indices_of_ones_in_left_graph[i]]]
#             #     print(f"This left  child fulfills: {left_child}")
#             #     print(child_states_left[left_child])
#             #     inner_sum_left += child_states_left[left_child]
            
#             # inner_sum_right= 0
#             # for right_child in child_states_right:
#             #   print("right child is: ",right_child)
#             #   print(f"indices of one in right graph: {state_string_right}")
#             #   for i in range(len(state_string_right)):
#             #     print("i: ",i)
#             #     no_violation = True
#             #     if right_child[int(state_string_right[i])] != "1":
#             #       no_violation = False
#             #   if no_violation:
#             #     inner_sum_right += child_states_right[right_child]
#             #     # running_sum_right += child_states_right[right_child[indices_of_ones_in_right_graph[i]]]
              
              
#             #   inner_sum_combined = inner_sum_left*inner_sum_right

#               #  no_violation = True
#               #  iteration = 0
#               #  while no_violation:
#               #     print(" in while loop")
#               #     if left_child[indices_of_ones_in_left_graph[iteration]] == "1":
#               #        running_sum_left += left_child[indices_of_ones_in_left_graph[iteration]]
#               #        iteration += 1
#               #     else:
#               #        no_violation = False
#             # for right_child in child_states_right:
#             #    no_violation = True
#             #    iteration = 0
#             #    while no_violation:
#             #       if right_child[indices_of_ones_in_right_graph[iteration]] == "1":
#             #          running_sum_right += right_child[indices_of_ones_in_right_graph[iteration]]
#             #          iteration +=1
#             #       else:
#             #          no_violation = False
#         print(f"")
#         print(f"For the state {state_string}, state inner accumulator= {state_inner_accumulator}")
#         # state_sum_accumulator += combined_value
#         # print(f"For the state {state_string}, the result is runing sum left: {running_sum_left} multiplyed by running irght {running_sum_right} = {running_sum_left*running_sum_right}")
#         new_states[state_string] = state_inner_accumulator
#         # left_vertices_to_saturate = []
#         # left_vertices_to_unsaturate = []
#         # for left_identity_indices in list(powerset(range(len(state_string)),0,len(state_string)+1)):
#         #   right_identity_indices = [x for x in state_string if x not in left_identity_indices]
#         #   for identity_index in left_identity_indices:
#         #     right_identity_indices.remove(identity_index)
#         #     if identity_index == "0":
#         #       left_vertices_to_unsaturate.append(state_string[identity_index])
#         #     else:
#         #       left_vertices_to_saturate.append(state_string[identity_index])

#         # for child_state_left in child_states_left:
#         #   for child_state_right in child_states_right:

#         # child_states_left[state_string] * child_states_right[state_string]
                      

#       # for subset_of_bag in powerset_of_current_bag:
#       #   print(f"subset of bag: {subset_of_bag}")
#       #   left_sum = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph_left,list(subset_of_bag),set(current_bag).difference(set(subset_of_bag)))
#       #   right_sum = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph_right,set(current_bag).difference(set(subset_of_bag)),list(subset_of_bag))
#       #   running_sum += left_sum * right_sum

#       # print("After two recursive calls at join node: ",current)
#       vertices_encountered = vertices_encountered_left_set.union(vertices_encountered_right_set)
#       # vertices_encountered.update(set(decomposition.nodes[current]["values"]))
#       print(f"Returning from introduce node {current}. New states: {new_states}")
#       return new_states, vertices_encountered
  
#     elif len(children) == 1:
#       assert len(children) == 1, f"Expected 1 child, got {len(children)}"
#       child_states, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition, children[0],current)
#       print(f"One child case. Current: {current} and child_states: {child_states}")
#       # print("After recursive call at single node: ", current)

#       child_bag = decomposition.nodes[children[0]]["values"]
#       current_bag = decomposition.nodes[current]["values"]
#       child_bag.sort()
#       current_bag.sort()
#       print(f"Child values: {child_bag}")
#       print(f"Current values: {current_bag}")
#       if len(child_bag) > len(current_bag):
#         print(f"We must be at a forget node. aggregating. Current node: {current} and child: {children[0]} ")
#         forgotten_element = list(set(child_bag) - set(current_bag))
#         assert len(forgotten_element) == 1, f"Forgotten element set should only contain one value, but it is: {forgotten_element}"
#         forgotten_element = forgotten_element[0]
#         index_of_forget_element = child_bag.index(forgotten_element)
#         new_states = {}
#         for state_string in child_states:
#           print("In forget node looping over child states. Current child state string: ",state_string)
#           new_string = state_string[:index_of_forget_element] + state_string[index_of_forget_element+1:]
#           try:
#             new_states[new_string] = new_states[new_string] + child_states[state_string]
#           except KeyError:
#              new_states[new_string] = child_states[state_string]


#         print("Returning these new states from forget node: ", new_states)
#         return new_states, vertices_encountered
#       else:
#         print("we must be at introduce node")
#         introduced_element = list(set(current_bag) - set(child_bag))
#         assert len(introduced_element) == 1, f"introduced element should be 1 element but is: {introduced_element}"
#         introduced_element = introduced_element[0]
#         index_of_introduced_element = current_bag.index(introduced_element)
#         print(f"Index of the element {introduced_element} within current values {current_bag} is {index_of_introduced_element}")
#         vertices_encountered.update(set(decomposition.nodes[current]["values"]))
#         # vertices_encountered.add(*decomposition.nodes[current]["values"])
#         # print(f"Inducing subgraph for these vertices: {vertices_encountered}")
#         induced_subgraph = graph.subgraph(vertices_encountered)
#         print(f"Induced subgraph nodes: {induced_subgraph.nodes}")
#         print(f"Induced subgraph edges: {induced_subgraph.edges}")
#         # matchings_in_induced_subgraph = count_matchings_simple(induced_subgraph)
#         # print(f"Matchings in induced subgraph at current node {current} is = {matchings_in_induced_subgraph}")

#         # count matchings in subgraph induced by nodes in vertices_encountered including current

#         # new_result = count_matchings_forcibly_saturate_vertex(induced_subgraph,introduced_element)
#         # print(f"count matchings included vertex: {current} result: {new_result} ")
#         # combined_result = new_result+sum_matchings_child_subgraph
#         new_states = {}
#         for state_string in child_states:
#           # print(f"STATE STRING IS: {state_string}")
#           # print("index of introduced element: ",index_of_introduced_element)
#           # Case where new node is NOT matched (append to string that new node is not matched and carry through the old value)
#           new_string_new_node_unsaturated = state_string[0:index_of_introduced_element] + "0" + state_string[index_of_introduced_element:]
#           # print(f"new node unsaturated. new string: {new_string_new_node_unsaturated} state_string: {state_string} child_states[state_string]: {child_states[state_string]}")
#           new_states[new_string_new_node_unsaturated] = child_states[state_string]



#           # Case where new node IS matched (append to string that new node is matched  and calculate in original graph)
#           new_string_new_node_saturated = state_string[0:index_of_introduced_element] + "1" + state_string[index_of_introduced_element:]
#           # print(f"new node sat. new string: {new_string_new_node_saturated} state_string: {state_string}")
#           # forcibly_saturate_result = count_matchings_forcibly_saturate_vertex(induced_subgraph,introduced_element)

#           child_elements_to_unsaturate = []
#           elements_to_saturate = [introduced_element]
#           print(f"Gonna construct child elements to unsaturate. statestring: {state_string}, child bag: {child_bag} els to sat: {elements_to_saturate} ")
#           for i in range(len(state_string)):
#             if state_string[i] == "0":
#               child_elements_to_unsaturate.append(child_bag[i])
#             else:
#               elements_to_saturate.append(child_bag[i])

#           forcibly_saturate_unsaturate_result = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,elements_to_saturate,child_elements_to_unsaturate)
#           print(f"sat/unsat result for the subgraph: {induced_subgraph.nodes} w edges: {induced_subgraph.edges} where els to sat are: {elements_to_saturate} and to unsat are: {child_elements_to_unsaturate}, result: {forcibly_saturate_unsaturate_result}")
#           # print(f"Calling forcibly saturated on this subgraph w nodes: {induced_subgraph.nodes} and forcibly matching element {introduced_element}. Result is: {forcibly_saturate_result}") 
#           # print(f"SETTING NEW STRING NODE SATURATED {new_string_new_node_saturated} TO VALUE {forcibly_saturate_unsaturate_result}")
#           new_states[new_string_new_node_saturated] = forcibly_saturate_unsaturate_result
#         # print("COMBINED RESULT: ",combined_result)
#         print("Returning from introduce node. New states: ",new_states)
#         return new_states, vertices_encountered

#     else:
#       raise Exception("Ey, too many children: ", len(children))
#   # After while loop, we are now at a leaf
#   print("THink we are at leaf, current: ",current)
#   leaf_value = decomposition.nodes[current]["values"]
#   # print(f"Leaf value: {leaf_value}")
#   # vertices_encountered.update(set(leaf_value))
#   # print("after adding initial to v encountered: ", vertices_encountered)
#   # for state in possible_states:
#   #   computation = 1+1+1
#   #   state_results[state] = computation
#   # return vertices_encountered
#   leaf_state = {"0":1, "1":0}
#   print(f"Returning leaf state: {leaf_state} and eaf value: {leaf_value}")
#   return leaf_state, set(leaf_value)
    

# def entry_count(graph, decomposition, decomposition_root):
#   state_results, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph,decomposition,decomposition_root,None)
#   number_of_matchings = 0
#   for key in state_results:
#     number_of_matchings += state_results[key]
#   return number_of_matchings

# Bit cleaner version

In [90]:
def evaluate_and_join_states(current_bag,states1,states2):
    total_sum = 0
    new_states = {}
    for pair_of_binary_strings in list(itertools.product(list(states1.keys()),list(states2.keys()))):
        if int(pair_of_binary_strings[0],2) & int(pair_of_binary_strings[1],2) == 0:
            # print(f"COMPATIBLE: {pair_of_binary_strings}")
            new_binary_string = bin(int(pair_of_binary_strings[0],2)+ int(pair_of_binary_strings[1],2))[2:]
            # new_state = ["1" if x == "1" else "0" for x in new_binary_string]
            while len(new_binary_string) < len(pair_of_binary_strings[0]):
                new_binary_string = "0"+new_binary_string
            inner_product = states1[pair_of_binary_strings[0]] * states2[pair_of_binary_strings[1]]
            # print("new bin string: ",new_binary_string)
            try:
                new_states[new_binary_string] += inner_product
            except:
                new_states[new_binary_string] = inner_product
            total_sum += inner_product
    return new_states, total_sum

In [91]:
def index_state_tables(states):
    indexed_states = {}
    for state in states:
        indexed_states[state] = (states[state],state.count("1"))
    return indexed_states

def convert_to_undefined_table(states):
    for state in list(states.keys()):
        states[state.replace("1","?")] = states.pop(state)
    return states



# Delete I believe. Think it was for where 1 = matched outside bag

In [92]:
def evaluate_and_join_states_fixed_join(current_bag,states1,states2):
    total_sum = 0
    new_states = {}
    for pair_of_binary_strings in list(itertools.product(list(states1.keys()),list(states2.keys()))):
        if int(pair_of_binary_strings[0],2) & int(pair_of_binary_strings[1],2) == 0:
            # print(f"COMPATIBLE: {pair_of_binary_strings}")
            new_binary_string = bin(int(pair_of_binary_strings[0],2)+ int(pair_of_binary_strings[1],2))[2:]
            # new_state = ["1" if x == "1" else "0" for x in new_binary_string]
            while len(new_binary_string) < len(pair_of_binary_strings[0]):
                new_binary_string = "0"+new_binary_string
            inner_product = states1[pair_of_binary_strings[0]] * states2[pair_of_binary_strings[1]]
            # print("new bin string: ",new_binary_string)
            try:
                new_states[new_binary_string] += inner_product
            except:
                new_states[new_binary_string] = inner_product
            total_sum += inner_product
    return new_states, total_sum

In [93]:
def count_states_forget_node(index_of_forgotten_element, child_states):

    # At a forget node, we simply aggregate the child states. 
    new_states = {}
    for state_string in child_states:
        # print("In forget node looping over child states. Current child state string: ",state_string)
        new_string = state_string[:index_of_forgotten_element] + state_string[index_of_forgotten_element+1:]
        try:
            new_states[new_string] = new_states[new_string] + child_states[state_string]
        except KeyError:
            new_states[new_string] = child_states[state_string]
    return new_states

def count_states_introduce_node(induced_subgraph, current_bag, child_bag, introduced_element, index_of_introduced_element, child_states):

    new_states = {}
    for state_string in child_states:
        # Case where new node is NOT matched (append to string that new node is not matched and carry through the old value)
        new_string_new_node_unsaturated = state_string[0:index_of_introduced_element] + "0" + state_string[index_of_introduced_element:]
        new_states[new_string_new_node_unsaturated] = child_states[state_string]



        # Case where new node IS matched (append to string that new node is matched  and calculate in original graph)
        new_string_new_node_saturated = state_string[0:index_of_introduced_element] + "1" + state_string[index_of_introduced_element:]

        child_elements_to_unsaturate = []
        elements_to_saturate = [introduced_element]
    #   print(f"Gonna construct child elements to unsaturate. statestring: {state_string}, child bag: {child_bag} els to sat: {elements_to_saturate} ")
        for i in range(len(state_string)):
            if state_string[i] == "0":
                child_elements_to_unsaturate.append(child_bag[i])
            else:
                elements_to_saturate.append(child_bag[i])
        forcibly_saturate_unsaturate_result = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,elements_to_saturate,child_elements_to_unsaturate)
        # print(f"sat/unsat result for the subgraph: {induced_subgraph.nodes} w edges: {induced_subgraph.edges} where els to sat are: {elements_to_saturate} and to unsat are: {child_elements_to_unsaturate}, result: {forcibly_saturate_unsaturate_result}")
        new_states[new_string_new_node_saturated] = forcibly_saturate_unsaturate_result
        new_string_new_node_saturated = state_string[0:index_of_introduced_element] + "1" + state_string[index_of_introduced_element:]
    return new_states




# Probably just delete. Was for the implementation where "1" means matched outside the bag.

In [94]:
# def count_states_forget_node_MATCHED_OUTSIDE(input_graph, current_bag, forgotten_element, index_of_forgotten_element, child_states):
#     induced_subgraph = input_graph.copy()
#     for pair in itertools.combinations(current_bag,2):
#         if induced_subgraph.has_edge(pair[0],pair[1]):
#             induced_subgraph.remove_edge(pair[0],pair[1])
    
#     # print(f"We must be at a forget node. aggregating. Current node: {current} and child: {children[0]} ")

#     # At the forget node, one element has been forgotten. This element is therefore "up for grabs" for participating in matchings at the 
#     # bag. We must forcibly saturate this element.
#     # For the new states where no elements in the bag are saturated, however, i.e. "0","00","000" etc.,
#     # we just propagate the corresponding child state
#     new_states = {}
#     for state_string in child_states:
#     #   print("In forget node looping over child states. Current child state string: ",state_string)
#         new_string = state_string[:index_of_forgotten_element] + state_string[index_of_forgotten_element+1:]
#         print(f"Evaluating new string: {new_string} under the child state string: {state_string} for forgotten el index: {index_of_forgotten_element}")
#         if new_string == "" or int(new_string,2) == 0:
#             try:
#                 new_states[new_string] = new_states[new_string] + child_states[state_string]
#             except:
#                 new_states[new_string] = child_states[state_string]

#         else:

#             # vertices_to_saturate = [vertices_to_saturate[m.start()] for m in re.finditer("1",state_string)]
#             # print(f"V to sat: {vertices_to_saturate}, v to unsat: {vertices_to_unsaturate} in G.n,e: {induced_subgraph.nodes} and {induced_subgraph.edges}")
#             # n_matchings = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,vertices_to_saturate,vertices_to_unsaturate)
#             # print(f"No. matchngs found: {n_matchings}")
#             try:
#                 new_states[new_string] = new_states[new_string] + child_states[state_string]
#             except KeyError:
#                 new_states[new_string] = child_states[state_string]
#     print(f"BEfore looping over new states, teh yare: {new_states}")
#     for new_state in new_states:
#         if "1" in new_state:
#             print(f"The state {new_state} contains 1")
#             n_of_ones = 0
#             indices_of_ones = []
#             for i in range(len(new_state)):
#                 if new_state[i] == "1":
#                     n_of_ones += 1
#                     indices_of_ones.append(i)
#             # This loop investigates ways to saturate the recently forgotten element.
#             # If, for example, we go from {u,v,w,x} -> {u,v,w} and forget node x,
#             # we must go over all ways to saturate x to one of the three remaining nodes.
#             # For example, in the new state "110", we have two options: the first 1 can saturate x,
#             # or the second 1 can saturate x. That is what I go over below, I think..
#             # The overall result for "110" is then the sum of numbers of matchings from the two 
#             # possibilities described above, PLUS the number of matchings in the child state for
#             # "110x" where x denotes the state that x had in the child, so: "1100" and "1101".
#             for i in range(n_of_ones):
#                 partner_of_forgotten_element = current_bag[indices_of_ones[i]]
#                 if input_graph.has_edge(partner_of_forgotten_element,forgotten_element) == False:
#                     print(f"Subgraph has no edge between {partner_of_forgotten_element} and {forgotten_element}")
#                     continue
#                 # Methodology to create an "island" of the forgotten element and its current partner
#                 induced_subgraph = induced_subgraph.copy()
#                 induced_subgraph.remove_node(forgotten_element)
#                 induced_subgraph.remove_node(partner_of_forgotten_element)
#                 induced_subgraph.add_node(forgotten_element)
#                 induced_subgraph.add_node(partner_of_forgotten_element)
#                 induced_subgraph.add_edge(partner_of_forgotten_element,forgotten_element)
#                 vertices_to_unsaturate = []
#                 other_vertices_to_saturate = []
#                 for j in range(len(new_state)):
#                     if new_state[j] == "0":
#                         vertices_to_unsaturate.append(current_bag[j])
#                     else:
#                         other_vertices_to_saturate.append(current_bag[j])
#                 # other_vertices_to_saturate = [current_bag[x] for x in indices_of_ones if indices_of_ones != partner_of_forgotten_element]
#                 print(f"Induced subgraph nodes: {induced_subgraph.nodes}. partner: {partner_of_forgotten_element} forgotten el: {forgotten_element} Other vertices to saturate: {other_vertices_to_saturate} v to unsat:{vertices_to_unsaturate}")
#                 num_matchings_local = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,[*other_vertices_to_saturate,forgotten_element],vertices_to_unsaturate)
#                 print(f"sum matchings local: {num_matchings_local}")
#                 new_states[new_state] = new_states[new_state] + num_matchings_local
#             # for child_state in child_states:
#             #     if child_state[:index_of_forgotten_element] + child_state[(1+index_of_forgotten_element):] == new_state:
#             #         num_matchings_local += child_states[child_state]
#             # new_states[new_state] = new_states[new_state] + num_matchings_local
                
#             # vertices_to_unsaturate = []
#             # # We must forcibly saturate the forgotten element - that's what is new at a forget node.
#             # vertices_to_saturate = [forgotten_element]
#             # for i in range(len(new_state)):
#             #     if new_state[i] == "1":
#             #         vertices_to_saturate.append(current_bag[i])
#             #     else:
#             #         vertices_to_unsaturate.append(current_bag[i])
#             # print(f"V to sat: {vertices_to_saturate}, v to unsat: {vertices_to_unsaturate} in G.n,e: {induced_subgraph.nodes} and {induced_subgraph.edges}")
#             # n_matchings = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,vertices_to_saturate,vertices_to_unsaturate)
#             # print(f"No. matchngs found: {n_matchings}")
#             # new_states[new_state] = new_states[new_state] + n_matchings

#     return new_states

# def count_states_introduce_node_MATCHED_OUTSIDE(induced_subgraph, current_bag, child_bag, introduced_element, index_of_introduced_element, child_states):
#     induced_subgraph = induced_subgraph.copy()
#     for pair in itertools.combinations(current_bag,2):
#         if induced_subgraph.has_edge(pair[0],pair[1]):
#             induced_subgraph.remove_edge(pair[0],pair[1])
    

#     # At an introduce node, we have introduced one new element, but this element may not participate in matchings between
#     # other elements within the bag, and vice versa. For new states where the introduced element has state 0, we just pass over the corresponding
#     # child state. For states where the introduced element is = 1, we must calculate the matchings that satisfy this identity in the original graph.
#     new_states = {}
#     for state_string in child_states:
#         # Case where new node is NOT matched (append to string that new node is not matched and carry through the old value)
#         new_string_new_node_unsaturated = state_string[0:index_of_introduced_element] + "0" + state_string[index_of_introduced_element:]
#         new_states[new_string_new_node_unsaturated] = child_states[state_string]



#         # Case where new node IS matched (append to string that new node is matched  and calculate in original graph)
#         new_string_new_node_saturated = state_string[0:index_of_introduced_element] + "1" + state_string[index_of_introduced_element:]

#         elements_to_unsaturate = []
#         elements_to_saturate = [introduced_element]
#     #   print(f"Gonna construct child elements to unsaturate. statestring: {state_string}, child bag: {child_bag} els to sat: {elements_to_saturate} ")
#         for i in range(len(state_string)):
#             if state_string[i] == "0":
#                 elements_to_unsaturate.append(child_bag[i])
#             else:
#                 elements_to_saturate.append(child_bag[i])
#         print(f"In INTRODUCE node, caling forcibly sat unsat with induced sugraph nodes: {induced_subgraph.nodes}")
#         print(f"els to sat: {elements_to_saturate} els to unsat: {elements_to_unsaturate}")
#         forcibly_saturate_unsaturate_result = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,elements_to_saturate,elements_to_unsaturate)
#         # print(f"vertices encountered: {vertices_encountered}")
#         print(f"sat/unsat result for the subgraph: {induced_subgraph.nodes} w edges: {induced_subgraph.edges} where els to sat are: {elements_to_saturate} and to unsat are: {elements_to_unsaturate}, result: {forcibly_saturate_unsaturate_result}")
#         new_states[new_string_new_node_saturated] = forcibly_saturate_unsaturate_result
#     return new_states




In [95]:
pcm_decomp.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [6, 7, 9]}, 2: {'values': [1, 2]}, 3: {'values': [0, 1]}, 4: {'values': [5, 6, 10]}, 5: {'values': [7, 8]}, 6: {'values': [4, 5, 10]}, 7: {'values': [1, 3]}, 8: {'values': [6, 9, 10]}, 9: {'values': [4]}, 10: {'values': [3]}, 11: {'values': [6, 9]}, 12: {'values': [7]}, 13: {'values': [0, 1, 3]}, 14: {'values': [5, 10]}, 15: {'values': [6, 10]}, 16: {'values': []}, 17: {'values': []}, 18: {'values': []}, 19: {'values': [0, 1, 3]}, 20: {'values': [0, 1, 3]}, 21: {'values': [1]}, 22: {'values': [1]}, 23: {'values': [0, 1]}, 24: {'values': [1]}, 25: {'values': [10, 4]}, 26: {'values': [6, 7]}, 27: {'values': [7]}})

In [278]:
def handle_common_edges_in_subgraphs(graph, decomposition, decomposition_root):
  def find_common_edges_in_subgraphs(graph, decomposition, current, parent):
    # for node in decomposition.nodes:
    #   print(node)
    #   decomposition[node]["edges_to_ignore"] = set()
    children = list(decomposition.neighbors(current))
    one_child_time_total = 0
    state_time_total = 0
    try:
      children.remove(parent)
    except ValueError:
      pass
      # print("Current element has no parent, must be root")
    # print(f"Current node: {current} has these children after removal of parent: {children}")
    while len(children) > 0:
      if len(children) == 2:
        # print(f"Handling common edges. at join node. Current: {current} and children: {children}")
        current_bag = decomposition.nodes[current]["values"]
        tree_vertices_encountered_left_set, graph_nodes_encountered_left = find_common_edges_in_subgraphs(graph, decomposition,children[0],current)
        tree_vertices_encountered_right_set, graph_nodes_encountered_right = find_common_edges_in_subgraphs(graph, decomposition,children[1],current)
        # print(f"JOIN\n tree vertices encounterede left, right: {tree_vertices_encountered_left_set}, {tree_vertices_encountered_right_set}\n Graph nodes enc left, right: {graph_nodes_encountered_left}, {graph_nodes_encountered_right}")
        
        # for tree_vertex_left in tree_vertices_encountered_left_set:
        #   print(f"Single tree vertex left set: {tree_vertex_left} its associated values: {decomposition.nodes[tree_vertex_left]['values']}")
        #   graph_nodes_in_left_subgraph = graph_nodes_in_left_subgraph + decomposition.nodes[tree_vertex_left]["values"]
        left_subgraph = graph.subgraph(graph_nodes_encountered_left)
        right_subgraph = graph.subgraph(graph_nodes_encountered_right)
        common_edges = []
        for edge in left_subgraph.edges:
          if right_subgraph.has_edge(edge[0],edge[1]):
            common_edges.append(edge)


        # edges_in_left_subgraph = set(graph.subgraph(graph_nodes_encountered_left).edges)
        # left_pre = graph.subgraph(graph_nodes_encountered_left)
        # print(f"nodes enc left: ")
        # print(f"left_pre_edges: {left_pre.edges}")
        # edges_in_right_subgraph = set(graph.subgraph(graph_nodes_encountered_right).edges)
        # common_edges = edges_in_left_subgraph.intersection(edges_in_right_subgraph)
        # print(f"After managing at join node. Nodes in left, right subgraph: {graph_nodes_in_left_subgraph}, {graph_nodes_in_right_subgraph}")
        # print(f"Still join. Edges in left, right: {edges_in_left_subgraph}, {edges_in_right_subgraph}")
        # print(f"Common edges: {common_edges}")
        if len(common_edges) > 0:
          for right_tree_node in tree_vertices_encountered_right_set:
            # print(f"right tree node: {right_tree_node} dc.nodes[rt](data=True): {decomposition.nodes[right_tree_node]}")
            decomposition.nodes[right_tree_node]["edges_to_ignore"] = decomposition.nodes[right_tree_node]["edges_to_ignore"].union(common_edges)
        tree_vertices_encountered = tree_vertices_encountered_left_set.union(tree_vertices_encountered_right_set)
        graph_nodes_encountered = graph_nodes_encountered_left.union(graph_nodes_encountered_right)
        return tree_vertices_encountered, graph_nodes_encountered
      elif len(children) == 1:
        assert len(children) == 1, f"Expected 1 child, got {len(children)}"
        tree_vertices_encountered, graph_nodes_encountered = find_common_edges_in_subgraphs(graph, decomposition, children[0],current)
        # print(f"in child = 1, tv enc: {tree_vertices_encountered} and gn enc: {graph_nodes_encountered}")
        tree_vertices_encountered.add(current)
        current_bag = decomposition.nodes[current]["values"]
        for el in current_bag:
          graph_nodes_encountered.add(el)
          
        # print(f"At current {current} returning tree V encountered: {tree_vertices_encountered} and g nodes encountered: {graph_nodes_encountered}")
        return tree_vertices_encountered, graph_nodes_encountered
      
    # After while loop, we are now at a leaf
    # print("THink we are at leaf, current: ",current)
    leaf_tree_vertices_encountered = set({current})
    # graph_nodes_encountered = set(decomposition.nodes[current]["values"])
    # print(f"Returning set of tree vertices encountered: {leaf_tree_vertices_encountered} and g nodes enc: {graph_nodes_encountered}")
    return leaf_tree_vertices_encountered, set()
  nx.set_node_attributes(decomposition, set({}), "edges_to_ignore")
  find_common_edges_in_subgraphs(graph,decomposition,decomposition_root,None)
  # print("Handling done")
  # print(decomposition.nodes(data=True))

In [276]:
def entry_count(graph, decomposition, decomposition_root,parent=None):
  state_results = {}

  def count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition, current, parent):
    children = list(decomposition.neighbors(current))
    one_child_time_total = 0
    state_time_total = 0
    try:
      children.remove(parent)
    except ValueError:
      pass
      # print("Current element has no parent, must be root")
    # print(f"Current node: {current} has these children after removal of parent: {children}")
    while len(children) > 0:
      if len(children) == 2:
        assert len(children) == 2, f"Expected 2 children, got {len(children)}"
        assert type(children) == list, f"TYpe of children should be list, it is {children}"
        current_bag = decomposition.nodes[current]["values"]
        child_states_left, vertices_encountered_left_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[0],current)
        child_states_right, vertices_encountered_right_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[1],current)
        """
        
        if vertices_encountered_left_set.issubset(vertices_encountered_right_set) or vertices_encountered_right_set.issubset(vertices_encountered_left_set):
          print(f"Ey whats going on, one subgraph is a susbet of the other? {vertices_encountered_left_set} and {vertices_encountered_right_set}")
          if len(vertices_encountered_left_set) > len(vertices_encountered_right_set):
            return child_states_left, vertices_encountered_left_set
          else:
            return child_states_right, vertices_encountered_right_set
        """
        # print("JOIN NODE: ",current)
        vertices_encountered_left = list(vertices_encountered_left_set)
        vertices_encountered_right = list(vertices_encountered_right_set)
        vertices_encountered_left.sort()
        vertices_encountered_right.sort()
        left_child_bag = decomposition.nodes[children[0]]["values"]
        right_child_bag = decomposition.nodes[children[1]]["values"]
        current_bag.sort()
        # print("Current bag: ",current_bag)
        left_child_bag.sort()
        right_child_bag.sort()
        # print(f"Child bag, states left: {left_child_bag, child_states_left} bag, states right: {right_child_bag, child_states_right}")

        # subgraph_left = graph.subgraph(vertices_encountered_left)
        # subgraph_right = graph.subgraph(vertices_encountered_right)
        # common_edges = set(subgraph_left.edges).intersection(set(subgraph_right.edges))
        # if len(common_edges):
        #   print(F"WARNING, COMMON EDGES IN SUBGRAPHS: {common_edges}")
        #   print(F"WARNING, COMMON EDGES IN SUBGRAPHS: {common_edges}")
        #   print(F"WARNING, COMMON EDGES IN SUBGRAPHS: {common_edges}")
        #   print(F"WARNING, COMMON EDGES IN SUBGRAPHS: {common_edges}")
        """
        for combination in list(itertools.combinations(current_bag,2)):
          if graph.has_edge(combination[0],combination[1]):
            print(f"WARNING: Graph has edge between these separator vertices: {combination}")
            print(f"WARNING: Graph has edge between these separator vertices: {combination}")
            print(f"WARNING: Graph has edge between these separator vertices: {combination}")
        """
        # # TODO
        # # TODO
        # # TODO
        # # TODO
        # # TODO
        # # I think that I can just literally use this method and delete all the other stuff below for counting states
        compat_states, compat_result = evaluate_and_join_states(current_bag,child_states_left,child_states_right)
        # print(f"comp states by function. result: {compat_result} for the states: {compat_states}")


        """
        # print(f"Current bag: {current_bag}")
        new_states = {}
        # Iterates over all possible states at the join node, e.g. "000", "001", "010", "011", etc
        time_state_strings_start = time.time_ns()
        for state_string_tuple in list(itertools.product("01",repeat=len(current_bag))):
          state_string = "".join(state_string_tuple)
          # print("STATE STRING: ",state_string)
          # Find the indices in the string which contain a 1
          # e.g. "010" -> [1],          "011" -> [1,2]
          indices_of_ones = []
          for i in range(len(state_string)):
              if state_string[i] == "1":
                  indices_of_ones.append(i)
          # print("INDIECS OF ONS: ",indices_of_ones)
          # Compute ways to distribute these ones to the two subgraphs,
          # e.g. [1] -> (([1],),(,[1])     [1,2] -> (([1,2],),([1],[2]),([2],[1]),(,[1,2]))
          state_inner_accumulator = 0
          for indices_of_ones_assigned_to_left in list(powerset(indices_of_ones,0,len(state_string)+1)):
              indices_of_ones_assigned_to_right = list(set(indices_of_ones)-set(indices_of_ones_assigned_to_left))
              # Now, for e.g. the state string "011" and the index example [1,2] -> ([1],[2]) 
              # this can be interpreted as "match index 1 of the state string in the left subgraph, match index 2 of the state string in the right subgraph"
              # I must now fill the "rest" of the state string indices with 0's in order to forcibly unsaturate them
              # then, I will obtain these signatures: subgraph1: "010" subgraph2: "001"
              # print(f"indiced of ones assigned to left: {indices_of_ones_assigned_to_left} and right: {indices_of_ones_assigned_to_right}")
              state_string_left = ["0" for x in range(len(state_string))]
              for index_of_one_to_left in indices_of_ones_assigned_to_left:
                  state_string_left[int(index_of_one_to_left)] = "1"
                  
              state_string_right = ["0" for x in range(len(state_string))]
              for index_of_one_to_right in indices_of_ones_assigned_to_right:
                  state_string_right[int(index_of_one_to_right)] = "1"
              # print(f"statestringleft: {state_string_left} and statesright: {state_string_right}")

              # print(f"Now ready to go, I think. join bag: {current_bag} bags left, right: {vertices_encountered_left} and {vertices_encountered_right}")
              # # print(f"And graph string left right: {graph_string_left} and {graph_string_right}")
              # print()
              # print(f"state string left graph: {state_string_left}")
              # print(f"state string right graph: {state_string_right}")
              # print()


              left_value = child_states_left["".join(state_string_left)]
              right_value = child_states_right["".join(state_string_right)]
              combined_value = left_value*right_value
              state_inner_accumulator += combined_value
          #     print("left value: ",left_value)
          #     print("right value: ",right_value)
          # print(f"")
          # print(f"For the state {state_string}, state inner accumulator= {state_inner_accumulator}")
          new_states[state_string] = state_inner_accumulator
          time_state_strings_stop = time.time_ns()
          # print()
          # print()
          # print(f"Time spent handling iterating over all state strings at join node: {(time_state_strings_start-time_state_strings_stop)/1000}")
          state_time_total += (time_state_strings_start-time_state_strings_stop)
        """


        # print(f"Before returning from join node {current}. VE left: {vertices_encountered_left}, VERight: {vertices_encountered_right}")
        vertices_encountered = vertices_encountered_left_set.union(vertices_encountered_right_set)


        # print(f"Returning from two children? node {current}. New states: {new_states}")
        # print(f"vertices encountered: {vertices_encountered}")
        # print(f"Returning from current join node: {current}, new states: {compat_states}")
        return compat_states, vertices_encountered
        # return new_states, vertices_encountered
      elif len(children) == 1:
        assert len(children) == 1, f"Expected 1 child, got {len(children)}"
        child_states, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition, children[0],current)

        child_bag = decomposition.nodes[children[0]]["values"]
        current_bag = decomposition.nodes[current]["values"]
        child_bag.sort()
        current_bag.sort()
        if len(child_bag) > len(current_bag):
          # print(f"FORGET NODE: {current} and child: {children[0]} ")
          forgotten_element = list(set(child_bag) - set(current_bag))
          assert len(forgotten_element) == 1, f"Forgotten element set should only contain one value, but it is: {forgotten_element}"
          forgotten_element = forgotten_element[0]
          index_of_forget_element = child_bag.index(forgotten_element)
          
          # print(f"Calling count states forget node for current: {current} with id of forg. {index_of_forget_element} and child states: {child_states}")
          new_states = count_states_forget_node(index_of_forget_element,child_states)
          # count_states_forget_node(graph.subgraph(vertices_encountered), current_bag,forgotten_element,index_of_forget_element,child_states)

          # print(f"Returning from forget node: {current}. New states: {new_states}, vertices encountered: {vertices_encountered}")
          return new_states, vertices_encountered
        else:
          # print(f"INTRODUCE NODE:{current} is introduce node")
          introduced_element = list(set(current_bag) - set(child_bag))
          assert len(introduced_element) == 1, f"introduced element should be 1 element but is: {introduced_element}"
          introduced_element = introduced_element[0]

          ### Ghetto way of handling the case where we are at the first introduce node after a leaf.
          if len(child_bag) == 0:
            # print(f"INTRODUCE: {current} is first node after leaf. Returning")
            return {"0":1,"1":0}, set({introduced_element})
          index_of_introduced_element = current_bag.index(introduced_element)
          vertices_encountered.update(set(decomposition.nodes[current]["values"]))
          induced_subgraph = graph.subgraph(vertices_encountered).copy()
          edges_to_ignore = decomposition.nodes[current]["edges_to_ignore"]
          # print(f"Current node: {current} with edges edges to inore: {edges_to_ignore} in induce dsub edges: {induced_subgraph.edges}")
          if len(edges_to_ignore) > 0:
            for edge in edges_to_ignore:
              # print(f"Evaluating edge: {edge} for induced subgraph edges: {induced_subgraph.edges}")
              if induced_subgraph.has_edge(edge[0],edge[1]):
                induced_subgraph.remove_edge(edge[0],edge[1])
                # print(f"Removed common edge: {edge}")


          new_states = count_states_introduce_node(induced_subgraph,current_bag,child_bag, introduced_element,index_of_introduced_element,child_states)
          # for state_string in child_states:
          #   # Case where new node is NOT matched (append to string that new node is not matched and carry through the old value)
          #   new_string_new_node_unsaturated = state_string[0:index_of_introduced_element] + "0" + state_string[index_of_introduced_element:]
          #   new_states[new_string_new_node_unsaturated] = child_states[state_string]



          #   # Case where new node IS matched (append to string that new node is matched  and calculate in original graph)
          #   new_string_new_node_saturated = state_string[0:index_of_introduced_element] + "1" + state_string[index_of_introduced_element:]

          #   child_elements_to_unsaturate = []
          #   elements_to_saturate = [introduced_element]
          # #   print(f"Gonna construct child elements to unsaturate. statestring: {state_string}, child bag: {child_bag} els to sat: {elements_to_saturate} ")
          #   for i in range(len(state_string)):
          #     if state_string[i] == "0":
          #       child_elements_to_unsaturate.append(child_bag[i])
          #     else:
          #       elements_to_saturate.append(child_bag[i])
          #   time_count = time.time()
          #   forcibly_saturate_unsaturate_result = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,elements_to_saturate,child_elements_to_unsaturate)
          #   time_count_stop = time.time()
          #   time_total = time_count_stop-time_count
          #   if time_total > 0.5:
          #     print(f"Time spent forcibly saturating counting: {time_total} seconds for node {current}")
          #     print(f"vertices encountered: {vertices_encountered}")
          #   print(f"sat/unsat result for the subgraph: {induced_subgraph.nodes} w edges: {induced_subgraph.edges} where els to sat are: {elements_to_saturate} and to unsat are: {child_elements_to_unsaturate}, result: {forcibly_saturate_unsaturate_result}")
          #   new_states[new_string_new_node_saturated] = forcibly_saturate_unsaturate_result
          # print(f"For vertices encountered: {vertices_encountered}")
          # one_child_time_stop = time.time_ns()
          # one_child_time_total += one_child_time_stop-one_child_time_start
          # # print(f"One child time spent: {round((one_child_time_stop-one_child_time_start)/1000)}")
          # print(f"Returning from introduce node: {current}. New states: {new_states} with v encountered: {vertices_encountered}")
          return new_states, vertices_encountered
      else:
        raise Exception("Ey, too many children: ", len(children))
    # After while loop, we are now at a leaf
    # print("THink we are at leaf, current: ",current)
    leaf_value = decomposition.nodes[current]["values"]
    # leaf_state = {"0":1, "1":0}
    leaf_state = {"0":1}
    # print(f"Returning leaf state: {leaf_state} and eaf value: {leaf_value}")
    # print(f"one child divide by 1000: {one_child_time_total}")
    # print(f"state total div by 1000: {state_time_total}")
    return leaf_state, set(leaf_value)
    


  handle_common_edges_in_subgraphs(graph,decomposition,decomposition_root)
  # print(f"In entry after handling: {decomposition.nodes(data=True)}")
  state_results, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph,decomposition,decomposition_root,parent)
  number_of_matchings = 0
  for key in state_results:
    number_of_matchings += state_results[key]
  return number_of_matchings

In [98]:
def construct_nice_decomp_and_count_matchings(graph):
    decomp, root = construct_nice_decomposition(graph)
    return entry_count(graph,decomp,root,None)

# Probably just delete this. Never got it to work fully (the forget node was tricky)

In [99]:
# state_results = {}

# def count_matchings_in_nice_tree_decomp_REMEMBER_STATE_1_means_matched_outside(graph, decomposition, current, parent):
#   children = list(decomposition.neighbors(current))
#   one_child_time_total = 0
#   state_time_total = 0
#   try:
#     children.remove(parent)
#   except ValueError:
#     pass
#     # print("Current element has no parent, must be root")
#   print(f"Current node: {current} has these children after removal of parent: {children}")
#   while len(children) > 0:
#     if len(children) == 2:
#       assert len(children) == 2, f"Expected 2 children, got {len(children)}"
#       assert type(children) == list, f"TYpe of children should be list, it is {children}"
#       current_bag = decomposition.nodes[current]["values"]
#       child_states_left, vertices_encountered_left_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[0],current)
#       child_states_right, vertices_encountered_right_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[1],current)
#       if vertices_encountered_left_set.issubset(vertices_encountered_right_set) or vertices_encountered_right_set.issubset(vertices_encountered_left_set):
#         print(f"Ey whats going on, one subgraph is a susbet of the other? {vertices_encountered_left_set} and {vertices_encountered_right_set}")
#         if len(vertices_encountered_left_set) > len(vertices_encountered_right_set):
#           return child_states_left, vertices_encountered_left_set
#         else:
#           return child_states_right, vertices_encountered_right_set
#       print("At join node. Current: ",current)
#       vertices_encountered_left = list(vertices_encountered_left_set)
#       vertices_encountered_right = list(vertices_encountered_right_set)
#       vertices_encountered_left.sort()
#       vertices_encountered_right.sort()
#       left_child_bag = decomposition.nodes[children[0]]["values"]
#       right_child_bag = decomposition.nodes[children[1]]["values"]
#       current_bag.sort()
#       print("Current bag: ",current_bag)
#       left_child_bag.sort()
#       right_child_bag.sort()
#       print(f"Child bag, states left: {left_child_bag, child_states_left} bag, states right: {right_child_bag, child_states_right}")

#       # subgraph_left = graph.subgraph(vertices_encountered_left)
#       # subgraph_right = graph.subgraph(vertices_encountered_right)
#       # common_edges = set(subgraph_left.edges).intersection(set(subgraph_right.edges))
#       # if len(common_edges):
#       #   print(F"WARNING, COMMON EDGES IN SUBGRAPHS: {common_edges}")
#       #   print(F"WARNING, COMMON EDGES IN SUBGRAPHS: {common_edges}")
#       #   print(F"WARNING, COMMON EDGES IN SUBGRAPHS: {common_edges}")
#       #   print(F"WARNING, COMMON EDGES IN SUBGRAPHS: {common_edges}")
      
#       for combination in list(itertools.combinations(current_bag,2)):
#         if graph.has_edge(combination[0],combination[1]):
#           print(f"WARNING: Graph has edge between these separator vertices: {combination}")
#           print(f"WARNING: Graph has edge between these separator vertices: {combination}")
#           print(f"WARNING: Graph has edge between these separator vertices: {combination}")
#       # # TODO
#       # # TODO
#       # # TODO
#       # # TODO
#       # # TODO
#       # # I think that I can just literally use this method and delete all the other stuff below for counting states
#       compat_states, compat_result = evaluate_and_join_states(current_bag,child_states_left,child_states_right)
#       print(f"comp states by function. result: {compat_result} for the states: {compat_states}")


#       print(f"Current bag: {current_bag}")
#       new_states = {}
#       # Iterates over all possible states at the join node, e.g. "000", "001", "010", "011", etc
#       time_state_strings_start = time.time_ns()
#       for state_string_tuple in list(itertools.product("01",repeat=len(current_bag))):
#         state_string = "".join(state_string_tuple)
#         # print("STATE STRING: ",state_string)
#         # Find the indices in the string which contain a 1
#         # e.g. "010" -> [1],          "011" -> [1,2]
#         indices_of_ones = []
#         for i in range(len(state_string)):
#             if state_string[i] == "1":
#                 indices_of_ones.append(i)
#         # print("INDIECS OF ONS: ",indices_of_ones)
#         # Compute ways to distribute these ones to the two subgraphs,
#         # e.g. [1] -> (([1],),(,[1])     [1,2] -> (([1,2],),([1],[2]),([2],[1]),(,[1,2]))
#         state_inner_accumulator = 0
#         for indices_of_ones_assigned_to_left in list(powerset(indices_of_ones,0,len(state_string)+1)):
#             indices_of_ones_assigned_to_right = list(set(indices_of_ones)-set(indices_of_ones_assigned_to_left))
#             # Now, for e.g. the state string "011" and the index example [1,2] -> ([1],[2]) 
#             # this can be interpreted as "match index 1 of the state string in the left subgraph, match index 2 of the state string in the right subgraph"
#             # I must now fill the "rest" of the state string indices with 0's in order to forcibly unsaturate them
#             # then, I will obtain these signatures: subgraph1: "010" subgraph2: "001"
#             # print(f"indiced of ones assigned to left: {indices_of_ones_assigned_to_left} and right: {indices_of_ones_assigned_to_right}")
#             state_string_left = ["0" for x in range(len(state_string))]
#             for index_of_one_to_left in indices_of_ones_assigned_to_left:
#                 state_string_left[int(index_of_one_to_left)] = "1"
                
#             state_string_right = ["0" for x in range(len(state_string))]
#             for index_of_one_to_right in indices_of_ones_assigned_to_right:
#                 state_string_right[int(index_of_one_to_right)] = "1"
#             # print(f"statestringleft: {state_string_left} and statesright: {state_string_right}")

#             # print(f"Now ready to go, I think. join bag: {current_bag} bags left, right: {vertices_encountered_left} and {vertices_encountered_right}")
#             # # print(f"And graph string left right: {graph_string_left} and {graph_string_right}")
#             # print()
#             # print(f"state string left graph: {state_string_left}")
#             # print(f"state string right graph: {state_string_right}")
#             # print()


#             left_value = child_states_left["".join(state_string_left)]
#             right_value = child_states_right["".join(state_string_right)]
#             combined_value = left_value*right_value
#             state_inner_accumulator += combined_value
#         #     print("left value: ",left_value)
#         #     print("right value: ",right_value)
#         # print(f"")
#         # print(f"For the state {state_string}, state inner accumulator= {state_inner_accumulator}")
#         new_states[state_string] = state_inner_accumulator
#         time_state_strings_stop = time.time_ns()
#         # print()
#         # print()
#         # print(f"Time spent handling iterating over all state strings at join node: {(time_state_strings_start-time_state_strings_stop)/1000}")
#         state_time_total += (time_state_strings_start-time_state_strings_stop)
#       print(f"Before returning from join node {current}. VE left: {vertices_encountered_left}, VERight: {vertices_encountered_right}")
#       vertices_encountered = vertices_encountered_left_set.union(vertices_encountered_right_set)
#       print(f"Returning from two children? node {current}. New states: {new_states}")
#       print(f"vertices encountered: {vertices_encountered}")
#       two_children_time_stop = time.time_ns()
#       # print(f"two children time: {(two_children_time_stop-two_children_time)}")
#       return new_states, vertices_encountered
#     elif len(children) == 1:
#       one_child_time_start = time.time_ns()
#       assert len(children) == 1, f"Expected 1 child, got {len(children)}"
#       child_states, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition, children[0],current)
#       # print(f"One child case. Current: {current} and child_states: {child_states}")

#       child_bag = decomposition.nodes[children[0]]["values"]
#       current_bag = decomposition.nodes[current]["values"]
#       child_bag.sort()
#       current_bag.sort()
#     #   print(f"Child values: {child_bag}")
#       print(f"Current bag: {current_bag}")
#       if len(child_bag) > len(current_bag):
#         print(f"We must be at a forget node. aggregating. Current node: {current} and child: {children[0]} ")
#         forgotten_element = list(set(child_bag) - set(current_bag))
#         assert len(forgotten_element) == 1, f"Forgotten element set should only contain one value, but it is: {forgotten_element}"
#         forgotten_element = forgotten_element[0]
#         index_of_forget_element = child_bag.index(forgotten_element)
        
#         new_states = count_states_forget_node(graph.subgraph(vertices_encountered), current_bag,forgotten_element,index_of_forget_element,child_states)

#         # new_states = {}
#         # for state_string in child_states:
#         # #   print("In forget node looping over child states. Current child state string: ",state_string)
#         #   new_string = state_string[:index_of_forget_element] + state_string[index_of_forget_element+1:]
#         #   try:
#         #     new_states[new_string] = new_states[new_string] + child_states[state_string]
#         #   except KeyError:
#         #      new_states[new_string] = child_states[state_string]


#         print(f"Returning these new states{new_states} from the forget node: {current} with vertices encountered: {vertices_encountered}")
#         return new_states, vertices_encountered
#       else:
#         print("we must be at introduce node")
#         introduced_element = list(set(current_bag) - set(child_bag))
#         assert len(introduced_element) == 1, f"introduced element should be 1 element but is: {introduced_element}"
#         introduced_element = introduced_element[0]
#         index_of_introduced_element = current_bag.index(introduced_element)
#         # print(f"Index of the element {introduced_element} within current values {current_bag} is {index_of_introduced_element}")
#         vertices_encountered.update(set(decomposition.nodes[current]["values"]))
#         # print(f"Induced subgraph nodes: {induced_subgraph.nodes}")
#         # print(f"Induced subgraph edges: {induced_subgraph.edges}")
#         new_states = count_states_introduce_node(graph.subgraph(vertices_encountered),current_bag,child_bag, introduced_element,index_of_introduced_element,child_states)
#         # for state_string in child_states:
#         #   # Case where new node is NOT matched (append to string that new node is not matched and carry through the old value)
#         #   new_string_new_node_unsaturated = state_string[0:index_of_introduced_element] + "0" + state_string[index_of_introduced_element:]
#         #   new_states[new_string_new_node_unsaturated] = child_states[state_string]



#         #   # Case where new node IS matched (append to string that new node is matched  and calculate in original graph)
#         #   new_string_new_node_saturated = state_string[0:index_of_introduced_element] + "1" + state_string[index_of_introduced_element:]

#         #   child_elements_to_unsaturate = []
#         #   elements_to_saturate = [introduced_element]
#         # #   print(f"Gonna construct child elements to unsaturate. statestring: {state_string}, child bag: {child_bag} els to sat: {elements_to_saturate} ")
#         #   for i in range(len(state_string)):
#         #     if state_string[i] == "0":
#         #       child_elements_to_unsaturate.append(child_bag[i])
#         #     else:
#         #       elements_to_saturate.append(child_bag[i])
#         #   time_count = time.time()
#         #   forcibly_saturate_unsaturate_result = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,elements_to_saturate,child_elements_to_unsaturate)
#         #   time_count_stop = time.time()
#         #   time_total = time_count_stop-time_count
#         #   if time_total > 0.5:
#         #     print(f"Time spent forcibly saturating counting: {time_total} seconds for node {current}")
#         #     print(f"vertices encountered: {vertices_encountered}")
#         #   print(f"sat/unsat result for the subgraph: {induced_subgraph.nodes} w edges: {induced_subgraph.edges} where els to sat are: {elements_to_saturate} and to unsat are: {child_elements_to_unsaturate}, result: {forcibly_saturate_unsaturate_result}")
#         #   new_states[new_string_new_node_saturated] = forcibly_saturate_unsaturate_result
#         print(f"Returning these new states {new_states} from introduce node {current} with v encountered: {vertices_encountered}")
#         # print(f"For vertices encountered: {vertices_encountered}")
#         # one_child_time_stop = time.time_ns()
#         # one_child_time_total += one_child_time_stop-one_child_time_start
#         # # print(f"One child time spent: {round((one_child_time_stop-one_child_time_start)/1000)}")
#         return new_states, vertices_encountered
#     else:
#       raise Exception("Ey, too many children: ", len(children))
#   # After while loop, we are now at a leaf
#   print("THink we are at leaf, current: ",current)
#   leaf_value = decomposition.nodes[current]["values"]
#   leaf_state = {"0":1, "1":0}
#   print(f"Returning leaf state: {leaf_state} and eaf value: {leaf_value}")
#   # print(f"one child divide by 1000: {one_child_time_total}")
#   # print(f"state total div by 1000: {state_time_total}")
#   return leaf_state, set(leaf_value)
    

# def entry_count(graph, decomposition, decomposition_root):
#   state_results, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph,decomposition,decomposition_root,None)
#   number_of_matchings = 0
#   for key in state_results:
#     number_of_matchings += state_results[key]
#   return number_of_matchings

In [100]:
forget_graph = nx.Graph()
forget_graph.add_nodes_from(["a","b","c"])
forget_graph.add_edge("a","b")
forget_graph.add_edge("b","c")

In [101]:
forget_decomp = nx.Graph()
forget_decomp.add_node(0,values=["a"])
forget_decomp.add_node(1,values=["a","b"])
forget_decomp.add_node(2,values=["b"])
forget_decomp.add_node(3,values=["b","c"])
forget_decomp.add_node(4,values=["c"])
forget_decomp.add_edge(0,1)
forget_decomp.add_edge(1,2)
forget_decomp.add_edge(2,3)
forget_decomp.add_edge(3,4)

In [102]:
join_graph = nx.Graph()
join_graph.add_nodes_from(["a","b","c","d"])
join_graph.add_edge("a","b")
join_graph.add_edge("b","c")
join_graph.add_edge("c","d")

join_decomp = nx.Graph()
join_decomp.add_node(0,values=["a"])
join_decomp.add_node(1,values=["a","b"])
join_decomp.add_node(2,values=["b"])
join_decomp.add_node(3,values=["b","c"])
join_decomp.add_node(4,values=["b","c"])
join_decomp.add_node(5,values=["b","c"])
join_decomp.add_node(6,values=["c"])
join_decomp.add_node(7,values=["c","d"])
join_decomp.add_node(8,values=["d"])
join_decomp.add_edge(0,1)
join_decomp.add_edge(1,2)
join_decomp.add_edge(2,3)
join_decomp.add_edge(3,4)
join_decomp.add_edge(4,5)
join_decomp.add_edge(5,6)
join_decomp.add_edge(6,7)
join_decomp.add_edge(7,8)

In [103]:
p3 = nx.path_graph(3)
p3_decomp, p3_root = construct_nice_decomposition(p3)
p5 = nx.path_graph(5)
p5_decomp, p5_root = construct_nice_decomposition(p5)
p6 = nx.path_graph(6)
p6_decomp, p6_root = construct_nice_decomposition(p6)
p7 = nx.path_graph(7)
p7_decomp, p7_root = construct_nice_decomposition(p7)
c3 = nx.cycle_graph(3)
c3_decomp, c3_root = construct_nice_decomposition(c3)
c4 = nx.cycle_graph(4)
c4_decomp, c4_root = construct_nice_decomposition(c4)
c5 = nx.cycle_graph(5)
c5_decomp, c5_root = construct_nice_decomposition(c5)
c6 = nx.cycle_graph(6)
c6_decomp, c6_root = construct_nice_decomposition(c6)
c7 = nx.cycle_graph(7)
c7_decomp, c7_root = construct_nice_decomposition(c7)
c20 = nx.cycle_graph(20)
c20_decomp, c20_root = construct_nice_decomposition(c20)

Making decomp nice on current: 3
Calling correct unnice from current. Only has 1 child: 3
Correcting the unnice node 3 with child: 0
Current: 3 is amnesiac
Making decomp nice on current: 5
Calling correct unnice from current. Only has 1 child: 5
Correcting the unnice node 5 with child: 0
Current node 5 has no violation with child 0. Recursing on child again
Making decomp nice on current: 0
Calling correct unnice from current. Only has 1 child: 0
Correcting the unnice node 0 with child: 2
Current node 0 has no violation with child 2. Recursing on child again
Making decomp nice on current: 2
Calling correct unnice from current. Only has 1 child: 2
Correcting the unnice node 2 with child: 1
Current node 2 has no violation with child 1. Recursing on child again
Making decomp nice on current: 1
Calling correct unnice from current. Only has 1 child: 1
Correcting the unnice node 1 with child: 4
calling handle eager introducer from introduce case. current, child: (1, 4)
Calling insert bridgin 

In [104]:
print(count_matchings_simple(c4))
entry_count(c4,c4_decomp,c4_root,None)

7
Current node: 8 is introduce node
Current node: 7 is introduce node
Returning from introduce node: 7. New states: {'00': 1, '10': 0, '01': 0, '11': 0} with v encountered: {1, 3}
Current node: 1 is introduce node
Returning from introduce node: 1. New states: {'000': 1, '100': 0, '010': 0, '110': 1, '001': 0, '101': 1, '011': 0, '111': 0} with v encountered: {0, 1, 3}
We must be at a forget node. aggregating. Current node: 2 and child: 1 
Returning from forget node: 2. New states: {'00': 1, '10': 1, '01': 1, '11': 0}, vertices encountered: {0, 1, 3}
Current node: 0 is introduce node
Returning from introduce node: 0. New states: {'000': 1, '010': 0, '100': 1, '110': 1, '001': 1, '011': 1, '101': 0, '111': 2} with v encountered: {0, 1, 2, 3}
We must be at a forget node. aggregating. Current node: 6 and child: 0 
Returning from forget node: 6. New states: {'00': 2, '01': 1, '10': 1, '11': 3}, vertices encountered: {0, 1, 2, 3}
We must be at a forget node. aggregating. Current node: 5 and 

7

In [105]:
print(count_matchings_simple(c5))
entry_count(c5,c5_decomp,c5_root,None)

11
Current node: 10 is introduce node
Current node: 9 is introduce node
Returning from introduce node: 9. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {3, 4}
Current node: 1 is introduce node
Returning from introduce node: 1. New states: {'000': 1, '100': 0, '010': 0, '110': 1, '001': 0, '101': 0, '011': 1, '111': 0} with v encountered: {2, 3, 4}
We must be at a forget node. aggregating. Current node: 4 and child: 1 
Returning from forget node: 4. New states: {'00': 1, '10': 1, '01': 1, '11': 0}, vertices encountered: {2, 3, 4}
Current node: 2 is introduce node
Returning from introduce node: 2. New states: {'000': 1, '100': 0, '010': 1, '110': 1, '001': 1, '101': 0, '011': 0, '111': 1} with v encountered: {1, 2, 3, 4}
We must be at a forget node. aggregating. Current node: 3 and child: 2 
Returning from forget node: 3. New states: {'00': 2, '10': 1, '01': 1, '11': 1}, vertices encountered: {1, 2, 3, 4}
Current node: 0 is introduce node
Returning from introduce n

11

In [106]:
for e in c5_decomp.nodes(data=True):
    print(e)

(0, {'values': [0, 1, 4], 'edges_to_ignore': set()})
(1, {'values': [2, 3, 4], 'edges_to_ignore': set()})
(2, {'values': [1, 2, 4], 'edges_to_ignore': set()})
(3, {'values': [1, 4], 'edges_to_ignore': set()})
(4, {'values': [2, 4], 'edges_to_ignore': set()})
(5, {'values': [], 'edges_to_ignore': set()})
(6, {'values': [], 'edges_to_ignore': set()})
(7, {'values': [0], 'edges_to_ignore': set()})
(8, {'values': [0, 1], 'edges_to_ignore': set()})
(9, {'values': [3, 4], 'edges_to_ignore': set()})
(10, {'values': [4], 'edges_to_ignore': set()})


In [107]:
c5_decomp.edges

EdgeView([(0, 3), (0, 8), (1, 4), (1, 9), (2, 3), (2, 4), (5, 7), (6, 10), (7, 8), (9, 10)])

In [108]:
entry_count(c20,c20_decomp,c20_root,None)
print(count_matchings_simple(c20))

Current node: 40 is introduce node
Current node: 39 is introduce node
Returning from introduce node: 39. New states: {'00': 1, '10': 0, '01': 0, '11': 0} with v encountered: {1, 19}
Current node: 11 is introduce node
Returning from introduce node: 11. New states: {'000': 1, '100': 0, '010': 0, '110': 1, '001': 0, '101': 1, '011': 0, '111': 0} with v encountered: {0, 1, 19}
We must be at a forget node. aggregating. Current node: 30 and child: 11 
Returning from forget node: 30. New states: {'00': 1, '10': 1, '01': 1, '11': 0}, vertices encountered: {0, 1, 19}
Current node: 8 is introduce node
Returning from introduce node: 8. New states: {'000': 1, '010': 0, '100': 1, '110': 1, '001': 1, '011': 0, '101': 0, '111': 1} with v encountered: {0, 1, 2, 19}
We must be at a forget node. aggregating. Current node: 29 and child: 8 
Returning from forget node: 29. New states: {'00': 2, '10': 1, '01': 1, '11': 1}, vertices encountered: {0, 1, 2, 19}
Current node: 9 is introduce node
Returning from 

In [109]:
entry_count(p3,p3_decomp,p3_root,None)

Current node: 6 is introduce node
Current node: 1 is introduce node
Returning from introduce node: 1. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {1, 2}
We must be at a forget node. aggregating. Current node: 2 and child: 1 
Returning from forget node: 2. New states: {'0': 1, '1': 1}, vertices encountered: {1, 2}
Current node: 0 is introduce node
Returning from introduce node: 0. New states: {'00': 1, '10': 0, '01': 1, '11': 1} with v encountered: {0, 1, 2}
We must be at a forget node. aggregating. Current node: 5 and child: 0 
Returning from forget node: 5. New states: {'0': 2, '1': 1}, vertices encountered: {0, 1, 2}
We must be at a forget node. aggregating. Current node: 3 and child: 5 
Returning from forget node: 3. New states: {'': 3}, vertices encountered: {0, 1, 2}


3

In [110]:
entry_count(p7,p7_decomp,p7_root,None)

Current node: 14 is introduce node
Current node: 5 is introduce node
Returning from introduce node: 5. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {5, 6}
We must be at a forget node. aggregating. Current node: 10 and child: 5 
Returning from forget node: 10. New states: {'0': 1, '1': 1}, vertices encountered: {5, 6}
Current node: 3 is introduce node
Returning from introduce node: 3. New states: {'00': 1, '10': 0, '01': 1, '11': 1} with v encountered: {4, 5, 6}
We must be at a forget node. aggregating. Current node: 7 and child: 3 
Returning from forget node: 7. New states: {'0': 2, '1': 1}, vertices encountered: {4, 5, 6}
Current node: 0 is introduce node
Returning from introduce node: 0. New states: {'00': 2, '10': 0, '01': 1, '11': 2} with v encountered: {3, 4, 5, 6}
We must be at a forget node. aggregating. Current node: 6 and child: 0 
Returning from forget node: 6. New states: {'0': 3, '1': 2}, vertices encountered: {3, 4, 5, 6}
Current node: 1 is introduc

21

In [111]:
count_matchings_simple(c4)

7

In [112]:
entry_count(c4,c4_decomp,c4_root,None)

Current node: 8 is introduce node
Current node: 7 is introduce node
Returning from introduce node: 7. New states: {'00': 1, '10': 0, '01': 0, '11': 0} with v encountered: {1, 3}
Current node: 1 is introduce node
Returning from introduce node: 1. New states: {'000': 1, '100': 0, '010': 0, '110': 1, '001': 0, '101': 1, '011': 0, '111': 0} with v encountered: {0, 1, 3}
We must be at a forget node. aggregating. Current node: 2 and child: 1 
Returning from forget node: 2. New states: {'00': 1, '10': 1, '01': 1, '11': 0}, vertices encountered: {0, 1, 3}
Current node: 0 is introduce node
Returning from introduce node: 0. New states: {'000': 1, '010': 0, '100': 1, '110': 1, '001': 1, '011': 1, '101': 0, '111': 2} with v encountered: {0, 1, 2, 3}
We must be at a forget node. aggregating. Current node: 6 and child: 0 
Returning from forget node: 6. New states: {'00': 2, '01': 1, '10': 1, '11': 3}, vertices encountered: {0, 1, 2, 3}
We must be at a forget node. aggregating. Current node: 5 and ch

7

In [113]:
c4_decomp.nodes(data=True)

NodeDataView({0: {'values': [1, 2, 3], 'edges_to_ignore': set()}, 1: {'values': [0, 1, 3], 'edges_to_ignore': set()}, 2: {'values': [1, 3], 'edges_to_ignore': set()}, 3: {'values': [], 'edges_to_ignore': set()}, 4: {'values': [], 'edges_to_ignore': set()}, 5: {'values': [1], 'edges_to_ignore': set()}, 6: {'values': [1, 2], 'edges_to_ignore': set()}, 7: {'values': [1, 3], 'edges_to_ignore': set()}, 8: {'values': [3], 'edges_to_ignore': set()}})

In [114]:
c4_decomp.edges

EdgeView([(0, 2), (0, 6), (1, 2), (1, 7), (3, 5), (4, 8), (5, 6), (7, 8)])

In [115]:
c4.edges

EdgeView([(0, 1), (0, 3), (1, 2), (2, 3)])

In [116]:
halicin_decomp, halicin_root = construct_nice_decomposition(halicin)

Making decomp nice on current: 22
Calling correct unnice from current. Only has 1 child: 22
Correcting the unnice node 22 with child: 2
Current: 22 is amnesiac
Making decomp nice on current: 27
Calling correct unnice from current. Only has 1 child: 27
Correcting the unnice node 27 with child: 2
Current node 27 has no violation with child 2. Recursing on child again
Making decomp nice on current: 2
Calling correct unnice from current. Only has 1 child: 2
Correcting the unnice node 2 with child: 16
The current values contain  at least one element not present in child values, and vice versa. This means that the current both forgets and introduces at the same time
Handling ambivalent case. calling handle eager introducer with current: 2 child: 16
Calling insert bridgin node with root: 2 and child: 16 with graph edges: [(0, 12), (0, 13), (1, 14), (1, 15), (2, 16), (2, 27), (3, 12), (3, 14), (4, 13), (4, 25), (5, 17), (5, 18), (6, 19), (6, 23), (7, 15), (7, 17), (8, 20), (8, 21), (9, 19), (9

In [117]:
entry_count(halicin,halicin_decomp,halicin_root,None)

Current node: 34 is introduce node
Current node: 6 is introduce node
Returning from introduce node: 6. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {9, 11}
We must be at a forget node. aggregating. Current node: 19 and child: 6 
Returning from forget node: 19. New states: {'0': 1, '1': 1}, vertices encountered: {9, 11}
Current node: 33 is introduce node
Returning from introduce node: 33. New states: {'00': 1, '01': 0, '10': 1, '11': 1} with v encountered: {9, 10, 11}
Current node: 9 is introduce node
Returning from introduce node: 9. New states: {'000': 1, '100': 0, '001': 0, '101': 0, '010': 1, '110': 1, '011': 1, '111': 0} with v encountered: {8, 9, 10, 11}
We must be at a forget node. aggregating. Current node: 20 and child: 9 
Returning from forget node: 20. New states: {'00': 2, '10': 1, '01': 1, '11': 0}, vertices encountered: {8, 9, 10, 11}
Current node: 8 is introduce node
Returning from introduce node: 8. New states: {'000': 2, '100': 0, '010': 1, '110'

1146

In [118]:
count_matchings_simple(halicin)

1146

In [119]:
aspirin_graph = read_smiles("O=C(C)Oc1ccccc1C(=O)O")
aspirin_decomp, aspirin_root = construct_nice_decomposition(aspirin_graph)

Making decomp nice on current: 19
Calling correct unnice from current. Only has 1 child: 19
Correcting the unnice node 19 with child: 2
Current: 19 is amnesiac
Making decomp nice on current: 30
Calling correct unnice from current. Only has 1 child: 30
Correcting the unnice node 30 with child: 2
Current node 30 has no violation with child 2. Recursing on child again
Making decomp nice on current: 2
Calling correct unnice from current. Only has 1 child: 2
Correcting the unnice node 2 with child: 15
The current values contain  at least one element not present in child values, and vice versa. This means that the current both forgets and introduces at the same time
Handling ambivalent case. calling handle eager introducer with current: 2 child: 15
Calling insert bridgin node with root: 2 and child: 15 with graph edges: [(0, 11), (0, 12), (1, 13), (1, 14), (2, 15), (2, 30), (3, 20), (3, 24), (4, 16), (4, 21), (5, 22), (5, 28), (6, 18), (6, 26), (6, 27), (7, 16), (7, 17), (8, 23), (8, 29), (9

In [120]:
handle_common_edges_in_subgraphs(aspirin_graph,aspirin_decomp,aspirin_root)

In [121]:
aspirin_decomp.nodes[2]
nx.set_node_attributes(aspirin_decomp,3,"edges_to_ignore")

In [122]:
for val in aspirin_decomp.nodes(data=True):
    print(val)

(0, {'values': [3, 4], 'edges_to_ignore': 3})
(1, {'values': [9, 10], 'edges_to_ignore': 3})
(2, {'values': [1, 2], 'edges_to_ignore': 3})
(3, {'values': [0, 1], 'edges_to_ignore': 3})
(4, {'values': [6, 7, 8], 'edges_to_ignore': 3})
(5, {'values': [10, 12], 'edges_to_ignore': 3})
(6, {'values': [5, 6, 9], 'edges_to_ignore': 3})
(7, {'values': [6, 8, 9], 'edges_to_ignore': 3})
(8, {'values': [10, 11], 'edges_to_ignore': 3})
(9, {'values': [1, 3], 'edges_to_ignore': 3})
(10, {'values': [4, 5, 9], 'edges_to_ignore': 3})
(11, {'values': [3], 'edges_to_ignore': 3})
(12, {'values': [4], 'edges_to_ignore': 3})
(13, {'values': [10, 11, 12], 'edges_to_ignore': 3})
(14, {'values': [9], 'edges_to_ignore': 3})
(15, {'values': [0, 1, 3], 'edges_to_ignore': 3})
(16, {'values': [6, 8], 'edges_to_ignore': 3})
(17, {'values': [6, 9], 'edges_to_ignore': 3})
(18, {'values': [5, 9], 'edges_to_ignore': 3})
(19, {'values': [], 'edges_to_ignore': 3})
(20, {'values': [], 'edges_to_ignore': 3})
(21, {'values'

In [123]:
entry_count(aspirin_graph,aspirin_decomp,aspirin_root,None)
entry_count(aspirin_graph,aspirin_decomp,aspirin_root)
print(count_matchings_simple(aspirin_graph))

Current node: 33 is introduce node
Current node: 3 is introduce node
Returning from introduce node: 3. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {0, 1}
Current node: 24 is introduce node
Returning from introduce node: 24. New states: {'000': 1, '001': 0, '100': 0, '101': 0, '010': 0, '011': 1, '110': 1, '111': 0} with v encountered: {0, 1, 3}
Current node: 38 is introduce node
Current node: 5 is introduce node
Returning from introduce node: 5. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {10, 12}
Current node: 28 is introduce node
Returning from introduce node: 28. New states: {'000': 1, '010': 0, '100': 0, '110': 1, '001': 0, '011': 0, '101': 1, '111': 0} with v encountered: {10, 11, 12}
Current node: 39 is introduce node
Current node: 8 is introduce node
Removed common edge: (10, 11)
Returning from introduce node: 8. New states: {'00': 1, '10': 0, '01': 0, '11': 0} with v encountered: {10, 11}
Current node: 29 is introduce node
Remov

In [124]:
benzene_graph = read_smiles("c1ccccc1")
benzene_decomp, benzene_root = construct_nice_decomposition(benzene_graph)

Making decomp nice on current: 7
Calling correct unnice from current. Only has 1 child: 7
Correcting the unnice node 7 with child: 1
Current: 7 is amnesiac
Making decomp nice on current: 9
Calling correct unnice from current. Only has 1 child: 9
Correcting the unnice node 9 with child: 1
Current: 9 is amnesiac
Making decomp nice on current: 10
Calling correct unnice from current. Only has 1 child: 10
Correcting the unnice node 10 with child: 1
Current node 10 has no violation with child 1. Recursing on child again
Making decomp nice on current: 1
Calling correct unnice from current. Only has 1 child: 1
Correcting the unnice node 1 with child: 4
Current node 1 has no violation with child 4. Recursing on child again
Making decomp nice on current: 4
Calling correct unnice from current. Only has 1 child: 4
Correcting the unnice node 4 with child: 0
Current node 4 has no violation with child 0. Recursing on child again
Making decomp nice on current: 0
Calling correct unnice from current. On

In [125]:
print(count_matchings_simple(benzene_graph))
entry_count(benzene_graph,benzene_decomp,benzene_root,None)

18
Current node: 12 is introduce node
Current node: 11 is introduce node
Returning from introduce node: 11. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {3, 4}
Current node: 2 is introduce node
Returning from introduce node: 2. New states: {'000': 1, '100': 0, '010': 0, '110': 1, '001': 0, '101': 0, '011': 1, '111': 0} with v encountered: {2, 3, 4}
We must be at a forget node. aggregating. Current node: 6 and child: 2 
Returning from forget node: 6. New states: {'00': 1, '10': 1, '01': 1, '11': 0}, vertices encountered: {2, 3, 4}
Current node: 3 is introduce node
Returning from introduce node: 3. New states: {'000': 1, '001': 0, '100': 1, '101': 0, '010': 1, '011': 1, '110': 0, '111': 1} with v encountered: {2, 3, 4, 5}
We must be at a forget node. aggregating. Current node: 5 and child: 3 
Returning from forget node: 5. New states: {'00': 2, '01': 1, '10': 1, '11': 1}, vertices encountered: {2, 3, 4, 5}
Current node: 0 is introduce node
Returning from introduce

18

In [126]:
entry_count(halicin,halicin_decomp,halicin_root)

Current node: 34 is introduce node
Current node: 6 is introduce node
Returning from introduce node: 6. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {9, 11}
We must be at a forget node. aggregating. Current node: 19 and child: 6 
Returning from forget node: 19. New states: {'0': 1, '1': 1}, vertices encountered: {9, 11}
Current node: 33 is introduce node
Returning from introduce node: 33. New states: {'00': 1, '01': 0, '10': 1, '11': 1} with v encountered: {9, 10, 11}
Current node: 9 is introduce node
Returning from introduce node: 9. New states: {'000': 1, '100': 0, '001': 0, '101': 0, '010': 1, '110': 1, '011': 1, '111': 0} with v encountered: {8, 9, 10, 11}
We must be at a forget node. aggregating. Current node: 20 and child: 9 
Returning from forget node: 20. New states: {'00': 2, '10': 1, '01': 1, '11': 0}, vertices encountered: {8, 9, 10, 11}
Current node: 8 is introduce node
Returning from introduce node: 8. New states: {'000': 2, '100': 0, '010': 1, '110'

1146

In [127]:
lsd = read_smiles("CCN(CC)C(=O)[C@H]1CN([C@@H]2Cc3c[nH]c4c3c(ccc4)C2=C1)C")
lsd_decomp, lsd_root = construct_nice_decomposition(lsd)

Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.


Making decomp nice on current: 39
Calling correct unnice from current. Only has 1 child: 39
Correcting the unnice node 39 with child: 3
Current: 39 is amnesiac
Making decomp nice on current: 53
Calling correct unnice from current. Only has 1 child: 53
Correcting the unnice node 53 with child: 3
Current: 53 is amnesiac
Making decomp nice on current: 54
Calling correct unnice from current. Only has 1 child: 54
Correcting the unnice node 54 with child: 3
Current node 54 has no violation with child 3. Recursing on child again
Making decomp nice on current: 3
Calling correct unnice from current. Only has 1 child: 3
Correcting the unnice node 3 with child: 28
Current node 3 has no violation with child 28. Recursing on child again
Making decomp nice on current: 28
Calling correct unnice from current. Only has 1 child: 28
Correcting the unnice node 28 with child: 19
Current node 28 has no violation with child 19. Recursing on child again
Making decomp nice on current: 19
Calling correct unnice

In [128]:
# entry_count(lsd, lsd_decomp,lsd_root,None)

In [129]:
# count_matchings_simple(lsd)

# 20 apr 17:55: Jeg er kommet til at ødelægge den "clean"  `count_matchings_vertex_included_multiple`, den som indeholder timing. den giver forkert resultat. fiks det lige.
# Resultat fra timingeksperimenter: over 90% af den lange tid bruges i de mange, mange kald til `is_matching`. Jeg tror ikke, at `nx.is_matching` er langsom i sig selv, men den lange tid skyldes, at `powerset` er *ENORMT*! Jeg skaber alle combinations, også selvom subsets af dem er invalid. F.eks., hvis edgesettet {(1,2),(3,4)} ikke er valid, så behøver jeg ikke at undersøge {(1,2),(3,4),(5,6)} osv... fiks dette !

# 24 apr 21:13 Okay er i gang med den nye procedure fra paper. Jeg tror at jeg har fikset forget node, så den gør det nye. Men introduce node gør også stadig noget (tæller matchings) - nu skal introduce node så vidt jeg ved bare være idle og ikke rigtig gøre noget men bare føre states videre rigth

# 26 apr 15:55: Jeg arbejder på Radus fix til join node. Jeg har nu implementeret at en introdude node tjekker om den aktuelt indeholder nogle af de edges, som den skal ignorere. Hvis den gør det, sletter den disse edges fra sin subgraph og tæller matchings. Det giver nu 172 i stedet for 335 for aspirin.

In [131]:
# mol = read_smiles("F[B-](F)(F)F.[H+]")
mol = read_smiles("CC(O)(P(=O)(O)O)P(=O)(O)O")

In [132]:
construct_nice_decomposition(mol)

Making decomp nice on current: 13
Calling correct unnice from current. Only has 1 child: 13
Correcting the unnice node 13 with child: 0
Current: 13 is amnesiac
Making decomp nice on current: 36
Calling correct unnice from current. Only has 1 child: 36
Correcting the unnice node 36 with child: 0
Current node 36 has no violation with child 0. Recursing on child again
Making decomp nice on current: 0
Calling correct unnice from current. Only has 1 child: 0
Correcting the unnice node 0 with child: 10
The current values contain  at least one element not present in child values, and vice versa. This means that the current both forgets and introduces at the same time
Handling ambivalent case. calling handle eager introducer with current: 0 child: 10
Calling insert bridgin node with root: 0 and child: 10 with graph edges: [(0, 10), (0, 36), (1, 12), (1, 28), (2, 14), (2, 34), (3, 15), (3, 30), (4, 16), (4, 35), (5, 17), (5, 24), (6, 18), (6, 32), (7, 19), (7, 33), (8, 20), (8, 26), (9, 11), (9

(<networkx.classes.graph.Graph at 0x1ccbe9a2990>, 13)

In [133]:
count_matchings_simple(mol)
dc, r = construct_nice_decomposition(mol)
entry_count(mol,dc,r,None)
# construct_nice_decomp_and_count_matchings(mol)

Making decomp nice on current: 13
Calling correct unnice from current. Only has 1 child: 13
Correcting the unnice node 13 with child: 0
Current: 13 is amnesiac
Making decomp nice on current: 36
Calling correct unnice from current. Only has 1 child: 36
Correcting the unnice node 36 with child: 0
Current node 36 has no violation with child 0. Recursing on child again
Making decomp nice on current: 0
Calling correct unnice from current. Only has 1 child: 0
Correcting the unnice node 0 with child: 10
The current values contain  at least one element not present in child values, and vice versa. This means that the current both forgets and introduces at the same time
Handling ambivalent case. calling handle eager introducer with current: 0 child: 10
Calling insert bridgin node with root: 0 and child: 10 with graph edges: [(0, 10), (0, 36), (1, 12), (1, 28), (2, 14), (2, 34), (3, 15), (3, 30), (4, 16), (4, 35), (5, 17), (5, 24), (6, 18), (6, 32), (7, 19), (7, 33), (8, 20), (8, 26), (9, 11), (9

56

In [ ]:
count_matchings_simple(mol)

56

In [134]:
import deepchem
tox21 = deepchem.molnet.load_tox21()
tasks, dataset, transformers = tox21
train, valid, test = dataset
import traceback

In [294]:
not_same = {}
it = 0
violation = False
# try:
for idx in train.ids:
    mol = read_smiles(idx)
    # print(idx)
    if mol.number_of_nodes()>2 and mol.number_of_nodes()<21:
        m_simple = count_matchings_simple(mol)
        m_entry = construct_nice_decomp_and_count_matchings(mol)
        it += 1
        if m_entry != m_simple:
            not_same[idx] = [m_simple,m_entry]
            violation = True
            # print(f"Not same for mol: {idx}. Simple: {m_simple}, decomp: {m_entry}")
    # print(read_smiles(idx).number_of_nodes())
    # print(idx)
    # if it > 10000:
    #     break
# except:
#     print("FAILED ID: ",idx)
#     traceback.print_exc()
print("it: ",it)
for p in not_same:
    print(p, not_same[p])
# print(not_same)
print(violation)

it:  4629
False


In [293]:
maxnum = 0
for idx in train.ids:
    mol = read_smiles(idx)
    mol = mol.number_of_nodes()
    if mol > maxnum:
        print(f"{mol} is greater than {maxnum}")
        maxnum = mol
print(maxnum)

11 is greater than 0
20 is greater than 11
21 is greater than 20
25 is greater than 21
33 is greater than 25
36 is greater than 33
37 is greater than 36
40 is greater than 37
57 is greater than 40
63 is greater than 57
74 is greater than 63
75 is greater than 74
85 is greater than 75
103 is greater than 85
103


In [234]:
def print_decomp(decomp,current,parent):
    children = list(decomp.neighbors(current))
    if parent != None:
        children.remove(parent)
    if len(children) > 1:
        try:
            print(f"JOIN current: {current}, vals: {decomp.nodes[current]['values']}, children: {children}, edges to ignore: {decomp.nodes[current]['edges_to_ignore']}")
        except KeyError:
            print(f"JOIN current: {current}, vals: {decomp.nodes[current]['values']}, children: {children}")

    else:
        try:
            print(f"current: {current}, vals: {decomp.nodes[current]['values']}, children: {children}, edges to ignore: {decomp.nodes[current]['edges_to_ignore']}")
        except KeyError:
            print(f"JOIN current: {current}, vals: {decomp.nodes[current]['values']}, children: {children}")

        # print(children)
    for child in children:
        print_decomp(decomp,child,current)
    print("BRANCH END")
    # neighbors = list(decomp.neighbors(list(decomp.neighbors(root))[0]))
    # if len(neighbors) > 1:
    #     print(neighbors)
    # print_decomp(decomp,neighbors[0])

In [150]:
m = read_smiles("Nc1cc(=O)nc(N)[nH]1")
for n in m.nodes(data=True):
    print(n)

(0, {'element': 'N', 'charge': 0, 'aromatic': False, 'hcount': 2})
(1, {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 0})
(2, {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 1})
(3, {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 0})
(4, {'element': 'O', 'charge': 0, 'aromatic': False, 'hcount': 0})
(5, {'element': 'N', 'charge': 0, 'aromatic': False, 'hcount': 0})
(6, {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 0})
(7, {'element': 'N', 'charge': 0, 'aromatic': False, 'hcount': 2})
(8, {'charge': 0, 'hcount': 1, 'aromatic': False, 'element': 'N'})


In [262]:
print(len(list(nx.connected_components(m))))
md1, mr1  = construct_junction_and_add_leaves(m)
print(len(list(nx.connected_components(md1))))
handle_promiscuous_nodes2(md1,mr1,None)
print(len(list(nx.connected_components(md1))))
handle_join_nodes(md1,mr1,None)
print(len(list(nx.connected_components(md1))))
# print_decomp(md1,mr1,None)
make_decomposition_nice(md1,mr1, None)
print(len(list(nx.connected_components(md1))))

1
1
1
1
Current current: 13 has no parent, is probably current of entire tree
Child is a join node. Must remove the parent instead
Removing current: 19 which has neigbors: [1, 17]
Adding edge between parent: 1 and child: 17
1


In [263]:
m_d, m_r = construct_nice_decomposition(m)

# for n in m_d.nodes(data=True):
#     print(n)
# handle_common_edges_in_subgraphs(m,m_d,m_r)
# for n in m_d.nodes(data=True):
#     print(n)
print_decomp(m_d,m_r,None)

Current current: 13 has no parent, is probably current of entire tree
Child is a join node. Must remove the parent instead
Removing current: 19 which has neigbors: [1, 17]
Adding edge between parent: 1 and child: 17
JOIN current: 13, vals: [], children: [22]
JOIN current: 22, vals: [3], children: [0]
JOIN current: 0, vals: [3, 4], children: [7]
JOIN current: 7, vals: [3], children: [23]
JOIN current: 23, vals: [8, 3], children: [1]
JOIN current: 1, vals: [3, 6, 8], children: [18, 17]
JOIN current: 18, vals: [3, 6, 8], children: [8]
JOIN current: 8, vals: [3, 6], children: [4]
JOIN current: 4, vals: [3, 5, 6], children: [24]
JOIN current: 24, vals: [5, 6], children: [25]
JOIN current: 25, vals: [6], children: [16]
JOIN current: 16, vals: [], children: []
BRANCH END
BRANCH END
BRANCH END
BRANCH END
BRANCH END
BRANCH END
JOIN current: 17, vals: [8, 3, 6], children: [20, 21]
JOIN current: 20, vals: [8, 3, 6], children: [9]
JOIN current: 9, vals: [3, 8], children: [5]
JOIN current: 5, vals:

In [239]:
list(nx.connected_components(m_d))

[{0,
  1,
  2,
  4,
  5,
  6,
  7,
  8,
  9,
  11,
  12,
  13,
  14,
  16,
  18,
  19,
  22,
  23,
  24,
  25,
  26,
  27},
 {3, 10, 15, 21}]

In [232]:
for e in m_d.edges:
    print(e)

(0, 7)
(0, 22)
(1, 18)
(1, 19)
(1, 23)
(2, 11)
(2, 27)
(3, 10)
(3, 15)
(4, 8)
(4, 24)
(5, 9)
(5, 12)
(6, 12)
(6, 26)
(7, 23)
(8, 18)
(9, 19)
(10, 21)
(11, 26)
(13, 22)
(14, 27)
(16, 25)
(24, 25)


In [235]:
print_decomp(m_d,m_r,None)

JOIN current: 13, vals: [], children: [22]
JOIN current: 22, vals: [3], children: [0]
JOIN current: 0, vals: [3, 4], children: [7]
JOIN current: 7, vals: [3], children: [23]
JOIN current: 23, vals: [8, 3], children: [1]
JOIN current: 1, vals: [3, 6, 8], children: [18, 19]
JOIN current: 18, vals: [3, 6, 8], children: [8]
JOIN current: 8, vals: [3, 6], children: [4]
JOIN current: 4, vals: [3, 5, 6], children: [24]
JOIN current: 24, vals: [5, 6], children: [25]
JOIN current: 25, vals: [6], children: [16]
JOIN current: 16, vals: [], children: []
BRANCH END
BRANCH END
BRANCH END
BRANCH END
BRANCH END
BRANCH END
JOIN current: 19, vals: [3, 6, 8], children: [9]
JOIN current: 9, vals: [3, 8], children: [5]
JOIN current: 5, vals: [2, 3, 8], children: [12]
JOIN current: 12, vals: [2, 8], children: [6]
JOIN current: 6, vals: [1, 2, 8], children: [26]
JOIN current: 26, vals: [1, 2], children: [11]
JOIN current: 11, vals: [1], children: [2]
JOIN current: 2, vals: [0, 1], children: [27]
JOIN current

In [264]:
entry_count(m,m_d,m_r,None)

Current element has no parent, must be root
Current node: 13 has these children after removal of parent: [22]
Current node: 22 has these children after removal of parent: [0]
Current node: 0 has these children after removal of parent: [7]
Current node: 7 has these children after removal of parent: [23]
Current node: 23 has these children after removal of parent: [1]
Current node: 1 has these children after removal of parent: [18, 17]
Current node: 18 has these children after removal of parent: [8]
Current node: 8 has these children after removal of parent: [4]
Current node: 4 has these children after removal of parent: [24]
Current node: 24 has these children after removal of parent: [25]
Current node: 25 has these children after removal of parent: [16]
Current node: 16 has these children after removal of parent: []
At current 25 returning tree V encountered: {16, 25} and g nodes encountered: {6}
At current 24 returning tree V encountered: {16, 25, 24} and g nodes encountered: {5, 6}
A

52

In [178]:
m.edges

EdgeView([(0, 1), (1, 2), (1, 8), (2, 3), (3, 4), (3, 5), (5, 6), (6, 7), (6, 8)])

In [216]:
handle_common_edges_in_subgraphs(m,m_d,m_r)

Current element has no parent, must be root
Current node: 13 has these children after removal of parent: [22]
Current node: 22 has these children after removal of parent: [0]
Current node: 0 has these children after removal of parent: [7]
Current node: 7 has these children after removal of parent: [23]
Current node: 23 has these children after removal of parent: [1]
Current node: 1 has these children after removal of parent: [18, 19]
Current node: 18 has these children after removal of parent: [8]
Current node: 8 has these children after removal of parent: [4]
Current node: 4 has these children after removal of parent: [24]
Current node: 24 has these children after removal of parent: [25]
Current node: 25 has these children after removal of parent: [16]
Current node: 16 has these children after removal of parent: []
At current 25 returning tree V encountered: {16, 25} and g nodes encountered: {6}
At current 24 returning tree V encountered: {16, 25, 24} and g nodes encountered: {5, 6}
A

In [217]:
for n in m_d.nodes(data=True):
    print(n)

(0, {'values': [3, 4], 'edges_to_ignore': set()})
(1, {'values': [3, 6, 8], 'edges_to_ignore': set()})
(2, {'values': [0, 1], 'edges_to_ignore': {(8, 6)}})
(3, {'values': [6, 7], 'edges_to_ignore': set()})
(4, {'values': [3, 5, 6], 'edges_to_ignore': set()})
(5, {'values': [2, 3, 8], 'edges_to_ignore': {(8, 6)}})
(6, {'values': [1, 2, 8], 'edges_to_ignore': {(8, 6)}})
(7, {'values': [3], 'edges_to_ignore': set()})
(8, {'values': [3, 6], 'edges_to_ignore': set()})
(9, {'values': [3, 8], 'edges_to_ignore': {(8, 6)}})
(10, {'values': [6], 'edges_to_ignore': set()})
(11, {'values': [1], 'edges_to_ignore': {(8, 6)}})
(12, {'values': [2, 8], 'edges_to_ignore': {(8, 6)}})
(13, {'values': [], 'edges_to_ignore': set()})
(14, {'values': [], 'edges_to_ignore': {(8, 6)}})
(15, {'values': [], 'edges_to_ignore': set()})
(16, {'values': [], 'edges_to_ignore': set()})
(18, {'values': [3, 6, 8], 'edges_to_ignore': set()})
(19, {'values': [3, 6, 8], 'edges_to_ignore': {(8, 6)}})
(21, {'values': [8, 3, 6

In [220]:
for e in m_d.edges:
    print(e)

(0, 7)
(0, 22)
(1, 18)
(1, 19)
(1, 23)
(2, 11)
(2, 27)
(3, 10)
(3, 15)
(4, 8)
(4, 24)
(5, 9)
(5, 12)
(6, 12)
(6, 26)
(7, 23)
(8, 18)
(9, 19)
(10, 21)
(11, 26)
(13, 22)
(14, 27)
(16, 25)
(24, 25)


In [218]:
entry_count(m,m_d,m_r,None)

Current element has no parent, must be root
Current node: 13 has these children after removal of parent: [22]
Current node: 22 has these children after removal of parent: [0]
Current node: 0 has these children after removal of parent: [7]
Current node: 7 has these children after removal of parent: [23]
Current node: 23 has these children after removal of parent: [1]
Current node: 1 has these children after removal of parent: [18, 19]
Current node: 18 has these children after removal of parent: [8]
Current node: 8 has these children after removal of parent: [4]
Current node: 4 has these children after removal of parent: [24]
Current node: 24 has these children after removal of parent: [25]
Current node: 25 has these children after removal of parent: [16]
Current node: 16 has these children after removal of parent: []
At current 25 returning tree V encountered: {16, 25} and g nodes encountered: {6}
At current 24 returning tree V encountered: {16, 25, 24} and g nodes encountered: {5, 6}
A

37

In [219]:
count_matchings_simple(m.subgraph([0,1,2,3,5,6,7,8]))

37

In [166]:
for n in m.nodes(data=True):
    print(n)

(0, {'element': 'N', 'charge': 0, 'aromatic': False, 'hcount': 2})
(1, {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 0})
(2, {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 1})
(3, {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 0})
(4, {'element': 'O', 'charge': 0, 'aromatic': False, 'hcount': 0})
(5, {'element': 'N', 'charge': 0, 'aromatic': False, 'hcount': 0})
(6, {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 0})
(7, {'element': 'N', 'charge': 0, 'aromatic': False, 'hcount': 2})
(8, {'charge': 0, 'hcount': 1, 'aromatic': False, 'element': 'N'})


In [167]:
for e in m.edges:
    print(e)

(0, 1)
(1, 2)
(1, 8)
(2, 3)
(3, 4)
(3, 5)
(5, 6)
(6, 7)
(6, 8)


In [ ]:
n1, w = construct_junction_and_add_leaves(a)
handle_promiscuous_nodes2(n1,w,None)
handle_join_nodes(n1,w,None)

In [ ]:
mold, molr = construct_junction_and_add_leaves(mol)
handle_promiscuous_nodes2(mold,molr,None)
handle_join_nodes(mold,molr,None)

In [ ]:
mold.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [1, 7]}, 2: {'values': [1, 2]}, 3: {'values': [7, 10]}, 4: {'values': [0, 1]}, 5: {'values': [3, 6]}, 6: {'values': [7, 9]}, 7: {'values': [7, 8]}, 8: {'values': [3, 5]}, 9: {'values': [1, 3]}, 10: {'values': [3, 6]}, 11: {'values': [1, 7]}, 12: {'values': [10, 7]}, 13: {'values': []}, 14: {'values': []}, 15: {'values': []}, 16: {'values': []}, 17: {'values': []}, 18: {'values': []}, 19: {'values': []}, 20: {'values': []}, 21: {'values': [1, 3, 5]}, 22: {'values': [0, 1, 2]}, 23: {'values': [8, 9, 7]}, 24: {'values': [3, 6]}, 25: {'values': [3, 6]}, 26: {'values': [1, 3, 5]}, 27: {'values': [1, 3, 5]}, 28: {'values': [1, 7]}, 29: {'values': [1, 7]}, 30: {'values': [10, 7]}, 31: {'values': [10, 7]}, 32: {'values': [8, 9, 7]}, 33: {'values': [8, 9, 7]}, 34: {'values': [0, 1, 2]}, 35: {'values': [0, 1, 2]}})

In [77]:
broken = read_smiles("CC(C)CCCC(C)(C)O")
count_matchings_simple(broken)

51

In [78]:
b_d, b_r = construct_nice_decomposition(broken)

Making decomp nice on current: 14
Calling correct unnice from current. Only has 1 child: 14
Correcting the unnice node 14 with child: 1
Current: 14 is amnesiac
Making decomp nice on current: 26
Calling correct unnice from current. Only has 1 child: 26
Correcting the unnice node 26 with child: 1
Current node 26 has no violation with child 1. Recursing on child again
Making decomp nice on current: 1
Calling correct unnice from current. Only has 1 child: 1
Correcting the unnice node 1 with child: 11
The current values contain  at least one element not present in child values, and vice versa. This means that the current both forgets and introduces at the same time
Handling ambivalent case. calling handle eager introducer with current: 1 child: 11
Calling insert bridgin node with root: 1 and child: 11 with graph edges: [(0, 9), (0, 10), (1, 11), (1, 26), (2, 15), (2, 24), (3, 9), (3, 13), (4, 16), (4, 25), (5, 17), (5, 20), (6, 13), (6, 22), (7, 18), (7, 23), (8, 10), (8, 12), (11, 20), (11

In [84]:
print_decomp(b_d,b_r,None)

KeyError: 'edges_to_ignore'

In [85]:
b_d.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [6, 9]}, 2: {'values': [1, 2]}, 3: {'values': [4, 5]}, 4: {'values': [0, 1]}, 6: {'values': [5, 6]}, 7: {'values': [6, 8]}, 8: {'values': [1, 3]}, 9: {'values': [4]}, 10: {'values': [3]}, 11: {'values': [6, 7]}, 12: {'values': [0, 1, 2]}, 13: {'values': [5]}, 14: {'values': []}, 15: {'values': []}, 16: {'values': []}, 17: {'values': []}, 18: {'values': []}, 19: {'values': [8, 5, 6]}, 20: {'values': [6, 7]}, 21: {'values': [6, 7]}, 22: {'values': [8, 5, 6]}, 23: {'values': [8, 5, 6]}, 24: {'values': [0, 1, 2]}, 25: {'values': [0, 1, 2]}, 26: {'values': [9]}, 27: {'values': [6]}, 28: {'values': [7]}, 29: {'values': [6]}, 30: {'values': [8, 6]}, 31: {'values': [1]}, 32: {'values': [0, 1]}, 33: {'values': [2]}, 34: {'values': [1]}, 35: {'values': [6]}})

In [86]:
entry_count(broken,b_d,b_r,None)

Current node: 28 is introduce node
Current node: 20 is introduce node
Returning from introduce node: 20. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {6, 7}
Current node: 33 is introduce node
Current node: 2 is introduce node
Returning from introduce node: 2. New states: {'00': 1, '10': 0, '01': 0, '11': 1} with v encountered: {1, 2}
Current node: 24 is introduce node
Returning from introduce node: 24. New states: {'000': 1, '100': 0, '010': 0, '110': 1, '001': 0, '101': 0, '011': 1, '111': 0} with v encountered: {0, 1, 2}
Current node: 34 is introduce node
Current node: 4 is introduce node
Removed common edge: (0, 1)
Returning from introduce node: 4. New states: {'00': 1, '10': 0, '01': 0, '11': 0} with v encountered: {0, 1}
Current node: 25 is introduce node
Removed common edge: (0, 1)
Removed common edge: (1, 2)
Returning from introduce node: 25. New states: {'000': 1, '001': 0, '100': 0, '101': 0, '010': 0, '011': 0, '110': 0, '111': 0} with v encountered: {

62